# Day 1: Inverse Captcha

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

- `1122` produces a sum of `3` (`1` + `2`) because the first digit (`1`) matches the second digit and the third digit (`2`) matches the fourth digit.
- `1111` produces `4` because each digit (all `1`) matches the next.
- `1234` produces `0` because no digit matches the next.
- `91212129` produces `9` because the only digit that matches the next one is the last digit, `9`.

What is the solution to your captcha?

In [37]:
def checksum_next(digits):
    return sum([int(digit) for (digit, next_digit) 
                in zip(digits, digits[1:]+digits[0]) 
                if digit == next_digit])

In [38]:
[checksum_next(digits) for digits in ["1122", "1111", "1234", "91212129"]]

[3, 4, 0, 9]

## Part Two

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit `10/2 = 5` steps forward matches it. Fortunately, your list has an even number of elements.

For example:

- `1212` produces `6`: the list contains `4` items, and all four digits match the digit `2` items ahead.
- `1221` produces `0`, because every comparison is between a `1` and a `2`.
- `123425` produces `4`, because both `2`s match each other, but no other digit has a match.
- `123123` produces `12`.
- `12131415` produces `4`.

In [24]:
def checksum_rotated(digits):
    offset = len(digits) // 2
    rotated_digits = digits[offset:] + digits[:offset]
    return sum([int(digit) for (digit, matching_digit) 
         in zip(digits, rotated_digits) 
         if digit == matching_digit])

In [25]:
[checksum_rotated(digits) for digits in ["1212", "1221", "123425", "123123", "12131415"]]


[6, 0, 4, 12, 4]

# Day 2: Corruption Checksum

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

```
5 1 9 5
7 5 3
2 4 6 8
```

- The first row's largest and smallest values are `9` and `1`, and their difference is `8`.
- The second row's largest and smallest values are `7` and `3`, and their difference is `4`.
- The third row's difference is `6`.

In this example, the spreadsheet's checksum would be `8 + 4 + 6 = 18`.


In [28]:
def checksum_spreadsheet(input):
    return sum([max(row) - min(row) for row in [[int(cell) for cell in line.split()] for line in input.split("\n")]])
    

In [31]:
checksum_spreadsheet("""5 1 9 5
7 5 3
2 4 6 8""")


18

## Part Two

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:

```
5 9 2 8
9 4 7 3
3 8 6 5
```

- In the first row, the only two numbers that evenly divide are `8` and `2`; the result of this division is `4`.
- In the second row, the two numbers are `9` and `3`; the result is `3`.
- In the third row, the result is `2`.

In this example, the sum of the results would be `4 + 3 + 2 = 9`.



In [77]:
def checksum_spreadsheet_divisible(input):
    return sum([b // a   for row in [sorted([int(cell) for cell in line.split()]) for line in input.split("\n")] for (a,b) in combinations(row, 2) if b % a == 0])

In [78]:
checksum_spreadsheet_divisible("""5 9 2 8
9 4 7 3
3 8 6 5""")

9

## Day 3: Spiral Memory

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

While this is very space-efficient (no squares are skipped), requested data must be carried back to square `1` (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square `1`.

For example:

- Data from square `1` is carried `0` steps, since it's at the access port.
- Data from square `12` is carried `3` steps, such as: down, left, left.
- Data from square `23` is carried only `2` steps: up twice.
- Data from square `1024` must be carried `31` steps.



In [326]:
from math import floor,ceil,sqrt

def dist(n):
    layer = floor(ceil(sqrt(n))/2)
    if (layer == 0):
        return 0
    
    square = (2*layer - 1) ** 2
    rest = n - square
    length = 2*layer
    side = ceil(rest / length) - 1
    middle = side * length + layer + square
    return abs(n - middle) + layer

[(a, dist(a)) for a in [1, 12, 23, 1024]]


[(1, 0), (12, 3), (23, 2), (1024, 31)]

## Part Two

So, the first few squares' values are chosen as follows:

- Square `1` starts with the value `1`.
- Square `2` has only one adjacent filled square (with value `1`), so it also stores `1`.
- Square `3` has both of the above squares as neighbors and stores the sum of their values, `2`.
- Square `4` has all three of the aforementioned squares as neighbors and stores the sum of their values, `4`.
- Square `5` only has the first and fourth squares as neighbors, so it gets the value `5`.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```


Lookup result in https://oeis.org/A141481/b141481.txt

# Day 4: High-Entropy Passphrases

A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:

- `aa bb cc dd ee` is valid.
- `aa bb cc dd aa` is not valid - the word `aa` appears more than once.
- `aa bb cc dd aaa` is valid - `aa` and `aaa` count as different words.



In [329]:
def is_valid_passphrase(phrase):
    words = phrase.split()
    return len(words) == len(set(words))

[phrase for phrase in ["aa bb cc dd ee",
                       "aa bb cc dd aa", 
                       "aa bb cc dd aaa"] 
 if is_valid_passphrase(phrase)]

['aa bb cc dd ee', 'aa bb cc dd aaa']

# Part Two

For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:

- `abcde fghij` is a valid passphrase.
- `abcde xyz ecdab` is not valid - the letters from the third word can be rearranged to form the first word.
- `a ab abc abd abf abj` is a valid passphrase, because all letters need to be used when forming another word.
- `iiii oiii ooii oooi oooo` is valid.
- `oiii ioii iioi iiio` is not valid - any of these words can be rearranged to form any other word.


In [343]:
def is_still_valid_passphrase(phrase):
    sorted_words = [" ".join(sorted(word)) for word in phrase.split()]
    return len(sorted_words) == len(set(sorted_words))
[phrase for phrase in ["abcde fghij",
                        "abcde xyz ecdab",
                        "a ab abc abd abf abj",
                        "iiii oiii ooii oooi oooo", 
                        "oiii ioii iioi iiio"]
if is_still_valid_passphrase(phrase)]

['abcde fghij', 'a ab abc abd abf abj', 'iiii oiii ooii oooi oooo']

# Day 5: A Maze of Twisty Trampolines, All Alike

An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

```
0
3
0
1
-3
```

Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

- `(0) 3  0  1  -3`  - before we have taken any steps.
- `(1) 3  0  1  -3`  - jump with offset `0` (that is, don't jump at all). Fortunately, the instruction is then incremented to `1`.
- `2 (3) 0  1  -3` - step forward because of the instruction we just modified. The first instruction is incremented again, now to `2`.
- `2  4  0  1 (-3)` - jump all the way to the end; leave a `4` behind.
- `2 (4) 0  1  -2`  - go back to where we just were; increment `-3` to `-2`.
- `2  5  0  1  -2`  - jump `4` steps forward, escaping the maze.

In this example, the exit is reached in `5` steps.

In [369]:
def steps(jumps):
    position = 0
    count = 0
    length = len(jumps)
    while 0 <= position < length:
        offset = jumps[position]
        jumps[position] = offset + 1
        position += offset
        count += 1
    return count
    
input = [0,3,0,1,-3]    
steps(input), input


(5, [2, 5, 0, 1, -2])

## Part Two

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by `1`. Otherwise, increase it by `1` as before.

Using this rule with the above example, the process now takes `10` steps, and the offset values after finding the exit are left as `2 3 2 3 -1`.


In [378]:
def crazy_steps(jumps):
    position = 0
    count = 0
    length = len(jumps)
    while 0 <= position < length:
        offset = jumps[position]
        jumps[position] = offset - 1 if offset >= 3 else offset + 1
        position += offset
        count += 1
    return count

input = [0,3,0,1,-3]
crazy_steps(input), input

(10, [2, 3, 2, 3, -1])

# Day 6: Memory Reallocation

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

- The banks start with `0`, `2`, `7`, and `0` blocks. The third bank has the most blocks, so it is chosen for redistribution.
- Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: `2 4 1 2`.
- Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: `3 1 2 3`.
- Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: `0 2 3 4`.
- The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: `1 3 4 1`.
- The third bank is chosen, and the same thing happens: `2 4 1 2`.

At this point, we've reached a state we've seen before: `2 4 1 2` was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is `5`.


In [24]:
import itertools

def countCycle(banks):
    count = 0
    seen = {}
    while tuple(banks) not in seen:
        seen[tuple(banks)] = count
        i, m = max(enumerate(banks), key=lambda k: (k[1], -k[0]))
        banks[i] = 0
        for j in itertools.islice(itertools.cycle(range(len(banks))), i + 1, i + m + 1):
            banks[j] += 1
        count += 1
    return count, count - seen[tuple(banks)]



In [26]:
countCycle([int(x) for x in "14	0	15	12	11	11	3	5	1	6	8	4	9	1	8	4".split()])[0]

11137

## Part Two

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, `2 4 1 2 `is seen again after four cycles, and so the answer in that example would be `4`.

In [27]:
countCycle([int(x) for x in "14	0	15	12	11	11	3	5	1	6	8	4	9	1	8	4".split()])[1]

1037

# Day 7: Recursive Circus

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

```
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
```
...then you would be able to recreate the structure of the towers that looks like this:

```
                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
```

In this example, `tknk` is at the bottom of the tower (the bottom program), and is holding up `ugml`, `padx`, and `fwft`. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

In [138]:
import re, collections

def readNodes(lines):
    weight = {}
    children = {}
    for line in lines:
        label, n, *xs = re.findall(r'\w+', line)
        weight[label] = int(n)
        children[label] = tuple(xs)
    return weight, children

In [141]:
lines="""pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)""".strip().splitlines()

In [158]:
weight, children = readNodes(lines)

root, = set(weight) - {c for cs in children.values() for c in cs}
root

'fbgguv'

## Part Two

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for `ugml`'s disc to be balanced, `gyxo`, `ebii`, and `jptl` must all have the same weight, and they do: `61`.

However, for `tknk` to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

- `ugml` + (`gyxo` + `ebii` + `jptl`) = 68 + (61 + 61 + 61) = 251
- `padx` + (`pbga` + `havc` + `qoyq`) = 45 + (66 + 66 + 66) = 243
- `fwft` + (`ktlj` + `cntj` + `xhth`) = 72 + (57 + 57 + 57) = 243

As you can see, `tknk`'s disc is unbalanced: `ugml`'s stack is heavier than the other two. Even though the nodes above `ugml` are balanced, `ugml` itself is too heavy: it needs to be `8` units lighter for its stack to weigh `243` and keep the towers balanced. If this change were made, its weight would be `60`.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?


In [157]:
import collections

def tree_weight(label):
    sub = [tree_weight(c) for c in children[label]]
    counter = collections.Counter(sub).most_common()
    if len(counter) == 2:
        (normal,_), (odd,_) = counter
        mutable = children[label][sub.index(odd)]
        print(weight[mutable] + (normal - odd))
        return weight[label] + sum(sub) + (normal - odd)
    else:
        return weight[label] + sum(sub)
    

In [159]:
tree_weight(root)

1864


537819

# Day 8: I Heard You Like Registers

You receive a signal directly from the CPU. Because of your recent assistance with jump instructions, it would like you to compute the result of a series of unusual register instructions.

Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:

```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```

These instructions would be processed as follows:

- Because a starts at `0`, it is not greater than `1`, and so `b` is not modified.
- `a` is increased by `1` (to `1`) because `b` is less than `5` (it is `0`).
- `c` is decreased by `-10` (to `10`) because `a` is now greater than or equal to `1` (it is `1`).
- `c` is increased by `-20` (to `-10`) because `c` is equal to `10`.

After this process, the largest value in any register is `1`.

You might also encounter `<=` (less than or equal to) or `!=` (not equal to). However, the CPU doesn't have the bandwidth to tell you what all the registers are named, and leaves that to you to determine.



In [207]:
def execute_program(input):
    registers = {}
    highest = 0
    conditions = {
        '<': lambda register, value: registers.get(register, 0) < value,
        '>': lambda register, value: registers.get(register, 0) > value,
        '<=': lambda register, value: registers.get(register, 0) <= value,
        '>=': lambda register, value: registers.get(register, 0) >= value,
        '==': lambda register, value: registers.get(register, 0) == value,
        '!=': lambda register, value: registers.get(register, 0) != value,
    }
    
    def execute(action, register, amount):
        if action == 'inc':
            registers[register] = registers.get(register, 0) + amount
        elif action == 'dec':
            registers[register] = registers.get(register, 0) - amount
    
    for line in input.strip().splitlines():
        target, action, amount, sep, check, comp, limit = re.findall(r'[^ ]+', line)
        amount, limit = int(amount), int(limit)
        if conditions[comp](check, limit):
            execute(action, target, amount)
        if registers.values():
            current = max(registers.values())
            highest = max(current, highest)
    return max(registers.values()), highest

In [208]:
input = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10"""
execute_program(input)

(1, 10)

## Part Two

To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations. For example, in the above instructions, the highest value ever held was 10 (in register c after the third instruction was evaluated).

# Day 9: Stream Processing

A large stream blocks your path. According to the locals, it's not safe to cross the stream at the moment because it's full of garbage. You look down at the stream; rather than water, you discover that it's a stream of characters.

You sit for a while and record part of the stream (your puzzle input). The characters represent groups - sequences that begin with `{` and end with `}`. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a `}` only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

Sometimes, instead of a group, you will find garbage. Garbage begins with `<` and ends with `>`. Between those angle brackets, almost any character can appear, including `{` and `}`. Within garbage, `<` has no special meaning.

In a futile attempt to clean up the garbage, some program has canceled some of the characters within it using `!`: inside garbage, any character that comes after `!` should be ignored, including `<`, `>`, and even another `!`.

You don't see any characters that deviate from these rules. Outside garbage, you only find well-formed groups, and garbage always terminates according to the rules above.

Here are some self-contained pieces of garbage:

- `<>`, empty garbage.
- `<random characters>`, garbage containing random characters.
- `<<<<>`, because the extra < are ignored.
- `<{!>}>`, because the first > is canceled.
- `<!!>`, because the second ! is canceled, allowing the `>` to terminate the garbage.
- `<!!!>>`, because the second ! and the first `>` are canceled.
- `<{o"i!a,<{i<a>`, which ends at the first `>`.

Here are some examples of whole streams and the number of groups they contain:

- `{}`, `1` group.
- `{{{}}}`, `3` groups.
- `{{},{}}`, also `3` groups.
- `{{{},{},{{}}}}`, `6` groups.
- `{<{},{},{{}}>}`, `1` group (which itself contains garbage).
- `{<a>,<a>,<a>,<a>}`, `1` group.
- `{{<a>},{<a>},{<a>},{<a>}}`, `5` groups.
- `{{<!>},{<!>},{<!>},{<a>}}`, `2` groups (since all but the last `>` are canceled).

Your goal is to find the total score for all groups in your input. Each group is assigned a score which is one more than the score of the group that immediately contains it. (The outermost group gets a score of `1`.)

- `{}`, score of `1`.
- `{{{}}}`, score of `1 + 2 + 3 = 6`.
- `{{},{}}`, score of `1 + 2 + 2 = 5`.
- `{{{},{},{{}}}}`, score of `1 + 2 + 3 + 3 + 3 + 4 = 16`.
- `{<a>,<a>,<a>,<a>}`, score of `1`.
- `{{<ab>},{<ab>},{<ab>},{<ab>}}`, score of `1 + 2 + 2 + 2 + 2 = 9`.
- `{{<!!>},{<!!>},{<!!>},{<!!>}}`, score of `1 + 2 + 2 + 2 + 2 = 9`.
- `{{<a!>},{<a!>},{<a!>},{<ab>}}`, score of `1 + 2 = 3`.


In [18]:
import re

def filter_stream(input):
    canceled = re.sub(r'!.', '', input)
    return re.sub(r'<[^>]*>', '', canceled)

def score(cleaned):
    level = 0
    score = 0
    for m in re.finditer(r'({*)(}*),?', cleaned):
        opened = len(m.group(1))
        closed = len(m.group(2))
        score += sum(range(1+level, 1+level+opened))
        level = level + opened - closed
    return score

[score(filter_stream(input)) for input in [
    "{}", 
    "{{{}}}",
    "{{},{}}",
    "{{{},{},{{}}}}",
    "{<a>,<a>,<a>,<a>}",
    "{{<ab>},{<ab>},{<ab>},{<ab>}}",
    "{{<!!>},{<!!>},{<!!>},{<!!>}}",
    "{{<a!>},{<a!>},{<a!>},{<ab>}}"]

]

[1, 6, 5, 16, 1, 9, 9, 3]

In [20]:
score(filter_stream('''{{{{{{{{{<}>,{<<'!!i}!>!!!!!>},<{,'!>,<!!e!!!>!!<a>}},{{{{<"o>}},{<!>,<!!!!o!ei"!}>}}},{}},{<},!!ua!!!{{!>,<!!!>},<o>}},{{{<o!>,<!!!>!>,>}}}},{{<,>,<>},{<'ui}<>,{{<!!!>,<,!,!>},<'}>,{<<!!}<<!!!!e!!!>}!!e>}},<e!!!>,!!!>ao'!"!!ooa!>},<"!!!>,<>}}},{{<i!>!>,<!>,<!u>},{{<oe!!<ii}i>,{<!>,<o}{{}a}!>},<!>},<!!!>!{!!!>!>},<}'u!o>}}},{{{<}!!!!!>},<!!e!>eu!>},<e>,<o!>,<!!!>"i!ao!>'}"eia,!>},<>},{}},{<<}!!!!a'!!!!!!a!!>}}}},{{{<i'!!!>!!!>!!!!i""!!u!!>}},{<!!!!o>,{}},{<!!"!<o"!>"{!oo!>,<!a>}},{{{<!!!!!<!!ou}>},<ee'!eee!!!>},<>},{},{<u{!>e!!au!!o!>},<!!!!!><'!!io'>}},{{{}},{{{<!!!!i,!!{}>}}}}},{{{{}},{{{{<<!!e{i"{,>},<!>},<i!!o<!!a'!!!!<e"a>},<!>e!!!>},<!a!!>}}},{{<!i}!>},<!!{i>,{}}}}}},{{{{{<!!iai!>},<!>},<,a!>>}}},{},{{{{{{{<!>},<a}<i{,{{!>},<u>}},<!>},<!!!>,<ua{!>},<!>},<!!{e}'ui>}},{{},{<u>}}}},{{{{<!!!!e!>,<}<e!>,<!!!!{i!a!!!>o"!!!>{>}},{{<ao!>},<e>},<!!!>,<!!!>e,!>!>},<u!!a,""{!!!>,<{>},{{{<!>},<'!!,!!'e"o!>},<!{,!>},<'u!"!"o'!!'>},{<!,!!!>!>},<!!<<!!!>},<{!>,<e!e{}!>},<!!<>,<i{!>{!>e"i!!!>{u"e!>},<a!>,<!>},<i>}},<!a!!!>!oo>}},{<<{i>,{{},{}}},{{{<!>,<<!>},<,,o{!>,<'!>},<u{!{}u!}!!!>},<>},{{{<!>!!!>},<}",!!<!!'>},<a!!!!!>}!e!!e!<u!!,''aa{!!o}'u>},<"'!!!><e<"!!<e!>,<i,'>}},<{!>},<e!!!>!>},<!o!!!!!>">}},{{},{<o!!ui!eia}!>,<oai>}},{{{{{{},{<'<<ao!o!>,<>}},<!!!!!>e!>!>},<!!!>!!'>},{<!!i'<!>,<!>!!}!>},<!!!>>,<!>},<!>},<!>,<<!>},<!!<!>'!>o!!u'!!!>i!>o,>}}}}},{{{{<o!!!>",a!,{!>},<',!!!>a!>"}!!!>},<>},{<a'!!!>,<<<<!>,<!!!!u!>,<!!{>}}}}},{{{},{<!u<!>},<>}},{{{{<uu!!!ai>,{<u"!>!!!>!!'i!e!!!>,<>,<!u'e!}oi,i}u<!>},<"}o"!!"'>}},{<!>,<eo}'<!!!!!>>,{}}},{{<>,{}},<o}u{e!!!!!>!>},<!!!>,<>}},{{{{<!!!>!>,<!>a,i'!>!,i!oeu!!!>,<!!!!!}"!>,<>},{<!{a,{!>,<{!>!!e'!,u!a!>!!!>o>,<!!!>!!!>o"<>}},{{<!u!!!>!>!!!!!>},<a!>a>}},{{{}},{{}}}},{{{{<"<o!>},<o'o!}{i!>}}>},<!"!>,<e!>!!!>!!iui!>},<,!!"a>},<!>},<u!!o!!!>!!'!,!!e!>}a!!"a!!e>},{{{}},{{<}o'!!'!"o!!au>},{{{<a!!u"!>'!!o!!!>""}'iui>}},{<!i'o{!!{!!!>!!}<!>,<o>}}}}},{{{<>},<o,iae!>,<{{<'<!>},<{!"!>},<!{>}}},{{{{<>}},{{{{},{<'!>!!ii!>},<!"o"a!>u>}},{{},{<,!>,<!>},<{{e'!!!!!>a'e"ei{!u!>,<!>},<!>>}}}}},{{}},{}},{{{<,u!!!>,<!!!>{<!!eo!!!>,<{!!",!ee!!!!>}},{{<e!!!>u!>,<e>},{<i,<!>},<ou!!>}},{{<!>,<'}<"!!'>},{<!!}!!!><!!!>'ea!!'!,!!!>'e!>},<"aae>,{<!>},<!ii'e"{a}o!>'!!o}"}>}}}}}},{{{<!>,<!!!>,<!!!><'!>e!!!>">,{<!!'!>,<{>,<,!a!>,<i<!!}"!!'u!!!>,<>}},{{},{<!>},<!>},<{,{!!!!""{!>,<!!,!!!>u'>}}},{{<!!!>,!!{!>,<<e{i{>}},{{{}},{{<!!!>}u<}!}!>},<!>},<<!>,<i}e>}},{<{'!!o}!i!>,<i!!u>,<}!>},<!!!>,<{!>},<a!ai'"!>,<oe}o!!!>o!>},<!!!>>}}}},{{{{{<}i!!a!>>},{<!>e!e}"!>>}},{{{{{<},!>},<au!>a>},{<,u"!!!a!ui>}},<{}'<oo!u!>a!>,!>},<u!!o'}!!!>>}},{<!>,<{!!!>},<!!!>!!!!!>,<"e!!!}"!!!!i!!!>,!!a,i!!!!!>},<>},{}},{<!!!>!>,<!o!!!!!!!!,!>},<'!>,<}>,{{<i!!'i,!>,<{'!!o!>,<!!!!"ii,!>i{">}}}},{{{{<i{>},<"u!,'">}},{{{{}},<!>,<!i"a>},{{},<o"!!!>!>},<!!!>>},{{<i!>},<o!!!>!>},<!!!>!>},<uo!!!!!!'aa!!!>e{!!a,o>}}},{},{{{{<<!},{!!!>a'}i!>},<!>},<<"!>},<!e!e!'>}},{<aiu'!>,<!"{!!o!!{!>,<,}>,<!o"!!<!">},{{<!!u!!!>},<!!!>!>},<au!!!>!>},<}!!"'i!>,<<i!>},<>}}},{{<"!>,!o!!>,{<!>,<e>}}}}},{{{{<'}<!>,<!!>}}}},{{{<!!iuo!!!>!>},<!'!!!>},<>,{<{!>},<!!"!<>}},{},{<>}},{{},{<"o!!!>!>,<'<'ua<}"!!!>,<!!!>!!{!!!>>}},{{{<!{!>},<!!!>i!>},<!!<>}},<'eui!ii!!o>}}},{{{{<!{!!!!!o>},{<eu!uuu}!!,!>},<,e{>,<!!}<!!!>!!e!!!>!!!au,!>,<{oio<>},{{<}e,'!>,<!!,!!!!}},!!!!!!!!!>!aou>,<e<a!a'!!!!!>o!}!>},<>}}},{{},{{<!!!>!!!>!>e}uie>}}},{{<e!!"!>,<!{{!!!>oi!>,<,!u>},{{<!!!>>},{<!>!>!!!>!>},<{!iee<a!'!!!!!><!!!!"!>},<'a>}}}},{{{{{<!>!!!>{{'oe<}a{}>,{<}!>},<!!!!!><!>,<o!!!>!!{o!e!>>}},{<a{,a!!}"{!!!!!!!!!!!!'i!!"!{<i,!>,<!u!>},<>,<{,!>,<!>!>},<!!!!i!!!>!!<e>}},{{<!!!>{!!>,{<o!!>}}},{{{<!>"o!!!>!,!!{!'"!>e!!!!u!!!!!!!a!>,<"!>},<>,<a"!!!>!>,<<,e!!!!{i>},{{{{},<<!!!><!!>},{{<!>,<o!>},<!,i!!!>},<<!>}!>,<!!!>>},{{<}i!!!>!!<{}!oe!!!>ie"!a{ia!>,<>},<!!!!oe!!!!}<!>},<o!!u"!!a>}}}},{{{{{},{<!<'!>eu!>,<e{'i}!!!>>,{{}}}},{<!>!>},<!"!!!>>},{{{<!!!!"u"!!!>!!!!!>!'!>,<>,{<!>},<!>},<!>},<!!!>!!!>!>!!!!u!!i!!<!!!>,!!{!>},<i>}},<'i'!a!o!!"!!,!!u{!{o{i'!!u>}}}}}}},{}},{{{{{<<!!!>}eu>}}},{{{},<'aa!!!>!>'}u>},{<}o!!'!,'<<<!!!<!>},<u}>,<!>},<"!!!>},<>},{{{<!>,<!{u!>},<>},<!!a!!!>o{}!!"'!e,'!!!!!>,"!>,<i{>},{<!!<!>},<,!!<!>},<>}}},{{<au!!!>!>,<oo!>,<a}u{'u!>},<!!'e!!a!>,<>,{<o!!oo"!!aua,!!!!eoiaea>}},{{{<ou!>,<oo}!>}!!!!!!{"}i!!!>,!ou{<>,<{u!>},<!>,<<!>!!!>},<!e!}<>}}},{{<!>!>,<i}a!!,}!!{!!!i!!u!>>,<}o!!>}}}},{{{<!!!>,<">},<!>,<''a'!!a!>!!!!a!>ia"!"!!!'{!>,<!>},<>},{<!}!!!>>,{<!!e>}},{{{{{{<e!>,<!!!>},<e!e!o!!',a!!!{'u{!e>}},{<o!>},<>},{{{<!>,<!>,<<!!!!!>o!>},<"!>,<io!>},<eoo,e{!i!u>},{{},{{<a,uu>}}}},{},{}}},{}}},{{{<i,>},{{}},{{<o!>},<u>},{{{{{},<,<o"}!!>}}},{}}}}}}},{{{<"!>},<!>,<!>},<ea!>},<!!{!!!!'i!>},<"!>,<""a>}},{<uo!!}eo!iua!i">,{{}}},{<!!!>a{!!!>,<<uo!>,<,e!>},<!"'a}>,<!>},<i!}o>}},{{{},{{<u"!>},<!>!!!!a{!>},<!>,<>}},{<!!',!!!!!a!>,<>,<'>}},{{{{<eia<<ua>},{{<>},{<!>,<!>},<!>},<{"i!>}">,{<!'!!"!>!>!>,<!!!>e>,{<!!ue{{{<'"ieu{<ee{>}}}}},{{<!{,!!!>!>},<!!!,!>,<!>{!!{>,{{{<!>},<!!u!>},<<!>},<!>},<!!ae<!!!>,<!>},<!'""{o>}}}},<'!>,<}{!!!!}e!>},<i!!!!!>'!>'}!>e>}},{{<"',e}a!!!oe!>},<!>,<!!!!!>!!!>,i{u<>},{<",a'u}<!!!>{!!a,{"'!!!>},<!>},<!>,<!!'>}}},{{}},{{{<},!!!>},<"!>},<'i!!!>u!!!>e!!}e"e>}}}}}},{},{{{{{},<u!!!!o!'}!!!>e'e!!e'!!,e!!!>!!<!!,>},{<!!!>i<!>}!!}}i<<!!,!!{>,{<>}},{{<i,a!!!,>}}},{{<!!!!,"o!},!!!>},<!!{e!ioo!>,<<u}!!'>},{<!>a!!!>"!>,<!!!>,<e!>{u{!!{!,<!!'>},{{},{<!>!>},<!!!>},<',,!!!!!>},<u,e!''!!!>}>},{{{<<<{!>,!>},<'{!!!>!>,<!!}!!{!!!!!>!>},<"}!!!!!>,<>},<!!!!>},{<{ae!!!>!!!}!!!>"!!!>},<a!!!>o}!>},<<!!!>>,{}}}}},{{{<o{'!>},<e"o!>!!uu!<e"i>}}}},{{{},{{{{},{{<u>},<{!><eea!!!!eue>},{<i!>!!!>},<!>,<!!>,{{{<!!!>!>,>},<}!>,!!!!,}e>}}}},{{{}},{},{{<!>},<!!!>"!!o!>,<{au>,<!!{"!{!">},<},e!!!!!!!>,<!!!'!!"!!{{'>}},{{{{{<!!{'!><'!!e!!!>},<>}}},{}},{{<{}!!ie<!>!>,<<u"!!!o>}},{{{<"!>,<u!!!>},<>},<!!ea'!>},<"a!>,<{!!!>},<!>},<!}!'!"!>>},{{},{<'!>!!!{!!,"e!>},<!!!>'>}},{{},{{<'ua!!<'u!!a}<}{'o>}}}}},{}}}},{{{{<!!!>,i!>!!!>,<!>a!>>},{<eo}a!>!i!>,<>}}},{{<!<{e,<{a'<i!>,<!>!!e!!!>aio!!<>,{<!,u<}!>},<}!!!>!>!!'!!}!>,<">}}},{{{<a{}!!!>'ia<!>},<e!>},<>}},<!>}"a!!!>!!>}},{{{<a!!{!!!>>},{<<!>},<,,<>}},{{}},{{{<i!!{i!>!i!!!>>}},{<!!{e!!!>}!>!!!!!!',!>,!!!>"a!!!>a"!>,<>},{<>,{{<{'>},<!!a""{!!!>!>,<{!>,<}">}}}},{{{<!!!>!!!"u}ae"'i!!}>,<!i!>a!!!>!!!!,!>i>},{{}},{{}}},{{{}},{{}},{<!e!>},<!>},<!>,<e<!!'>,{<,"!!,!!!>!>'!!!>'>}}},{{<!!a,o!u,o{o{o}}e>},<!>,<,,o"{e"!i'!!!!!>,<!!u!!!!!><>}}},{{{{{{}}},<!>,<!!!>,}u!!{!!!>},<!"{!'<!>,<!i>},{{<!!!>e!!!>u!!o>},{{<i,>}}},{<a!>,<{!>!>},<{!!u<<>,{{{<!>!!!>,<!o}!>,<"!}>}},<!!!>!>},<!!!,''!!au}!!!>!>!>,<">}}},{{},{{<!!!>u!!ea>}}},{{{<{,}'}!!e!>!!,'<!!!!'"!o>},{<!>,<!>,<a!>,!>!!!i!>>,<eoo!i}!>,<}"<e!>!>},<o}!>,<}!>e>},{<!>,<u!!!>},<"!>!!!>},<>,{<!>ao>}}},{{{},{{{{<>}},{},{}},{{<!>},<a!!!>!>,<,ea!!aa{i!>"<>},<!>,<a'ue<!!!>},<>},{{},{{},{{<!!!!"u!!o<'i"!!ou<i>},{<"!!!>"i!!'!!,u!>},<a'!}}a>}}}}},{<o!!!!!!u!!'!>,<!!!!a!u!>},<'a,!!!!!>o!!!>{!>,<!!i>}},{{{{{<<>},{{<!!!>!!'!!!>'i!!{!>!>,<,u>}},{}},{{<o!!!>}i'>,{<'!'e{i,"<!>,<}>}},{<!!e!!,e{i!!i!>},<i!!i!<!>},<!>},<e!!!><!!">,{<!!,e>}},{<}"!>>}},{{{{<!>,<a}!!o"eae<!!,!<!>,<'a>},{<}"'u!!!>,<a!>},<e>}},<!!{{!!o,,!!i!>},<ii!<'<}'i{!!>}},{{{<!!!>u}!!!>!!!!}'}aa!!u!!!>>}},{},{{<"!!>,<a>},{{<'!!!!!u{!><!!!>!>,<,">},{<!>!>!>,<>}}}}},{{<i!>},<i,"!!!>>,<!e,{i,i!o<!>},<a!{e!>,<>}},{{{<!>,<<<>}},{{},{<'!>},<!!!>},<{!>,<}!!a}oe!a'o!{>}},{<!!o}!>!>>}},{{{<!>,<!>,<a",!>,<a!>!!!>>,{<!>,<!>,<i!!!aau}!>},<e<!>,<>,{}}}},{{{<!>},<!!a!!,!>"i!!!!!!!>,!e!>,<o'!!!!!>"<>},{<!>},<!!,!>},<!>,<"!!!>},<o>}},<}'!ao!>e}>},{{<!!!>i!>{!>,<>}}}},{{<!>,<'}{"<!>,<'!!!>,<!>,<>,<!>u}!!"!aa!>,<{!>!"!>},<a!>},<!!!>,<>},{{{}},<!!<!!!>!"!!!!"!!e>}}}},{{<a}!!!!!!!>!<>},{{{<!>},<ee!>a<>},{{<<i!'"ua!>},<!>},<,!!!>!>,<,!>},<>},{{<!!!!i!>{!!!i!!!!!>!'a!!!>o!!{oo!>},<>}}}},{{<!>}!>,!!!><,i{o"!eauue>}},{<"!>u!!{<!i"!!}}!>},<!!a{>}},{}}}},{{},{{<iu"{!>,<!!{!>!{o!!"}!!!!'!e{>,{<!!!!!!!>!!!>,<!!!>"!!'!!""!!!!!>,!>,<,!>},<',}!!'!>,<>}},{{{{<!!'>,<>},{{<ou<ua>,<!,!!>},{<o}o!!a"}io>}},{{{{<aeo!,!>,<}'!!!>ei!>,<<!>,<oa>}},<eo<ie"!>},<!uu'!!!><,}"!>},<>}}},{<i!!!>!!!>ia<iu!!{!>},<>}},{{<u!o!!"iu{!>,<e{!!!>u!>},<>},<!>,<!'}}}!>,<!ue'e!!}>}},{<a}a!!}"i,,!>>}}}},{{{{<!!!>},<>},{<a!>},<<,!e{!>,<<}o!!'i!>,<,!>,<!!!>,>}},{{<<<'!}i!!!>}a!!!!!>!>,<,u,!!!>!!!>>},{{<>},{}}},{{<!>},<<!!!,u,{!!!>}!!'!i!!'>,<ai!!<io!>},<!!u<>},{{{{}},{<}a!!!>}a!!}!!!!!!"a>}},{}}}},{{{{}}},{<!,,!>{!>eeoa!!e>,<!>!!,{"{!<"}a<a!}ei,!a>},{<<!>!{!iu!>,<uaa>,<!!,e>}}}},{{{{},{<"!}!"!>},<!!!!!>,<!aa<!u{"!>{u"">,<iuoe!>},<<!>,<i!!<o!>,<u<e!!!!}>},{{<!!{{!!!>!!!>!>,<!!!>},<!>,<!!!>>,<!au{!!"i'>}}},{{},<"u<iu<a>}},{{<{!>},<i"},{ue!>u!!!>i{!>,<}>,<"i<!>},<u'a>}},{{{{<,}<o!!u!!!>>},{{<!!!><!!,{i{!"!!,'<'ou!u'!!"!!a>}}},{},{<!!!>!!e!!'!!}!><i'!!!>},<!!a!>},<{!!!>u!>,<u>,<!u!>!!}!!!>},<>}},{<i>,{}},{{{<<i!!ea!!!u,}!!!>u{!>!>,<<"u}>}},{{<e{<!!!>iaui,i}ou'{!<}>,{{<i!>!>},<!>,<"<!!!>!!"!,eu,!>,<>}}},{<>,{<<!!!>,<!>},<!>},<'"!!!>!!e>}}},{{<!>},<}!!<'!>},<!!!!!>"!!!!{!>,<,>},<!!!>},<!!!>u!>,<'"!e!!!>!!!>!u>}}}}},{{{<""'a!!!>!!'ue{u"ui'!>,<>,<<ea!!!!!>,<!!}aa,i{,!>,<>},{{<!!!'!>!!!><!!u,o!>>},{}},{{{{{<!>},<<!u,!!!e<i<!}o{>}}}}}}}},{{{{},{<oa"{<!>,<'}!!}e!!!>'>,<!>,<"!!!>,<!>,<e',u!!,,{o!!!>o,>},{{<!!!>a"!!"!!!>},<},i<<,e!>ii<>}}},{{{},{}},{<>,{}},{{<{!!!!}>}}},{{{{},{}},{{{},{{<!!!>,<!!!>},!>,<i!!!!!!!>{!>},<e}>},<!i!!{!{!>!!a!!!,<{!>,<{'e<a',i>},{}}}},{{{{{<!>},<{e"!!<!>},<a{e<>,{<,!!!>!e"o<,"<i"}!>},<uiu}>}},{<a'!!!!!,u!"!!!,}!>e!!!<ui'!>!!{a>}},{{{{<u!!!>!!!>"<"e!>',o!!!>u!>!!!!o!}>},{<!>},<{'eo>}},{{{},{<{i!a<a!!!'!!a!i>}},{{<!>,<{,}!!!>!ee}!uo},e,e!>},<u}>},{<!!!!!>,<"<u<!!!>!!'"ea!!,!!!!!!!>,<<a!!a>}}},{<ou'!>,<>}},{{<!i''o>},{<"}{e!>},<!!!>o!>},<>}},{}},{<!>},<{i{<!"!!!!<eo!!iu!!,!>>,{<e!>>}}},{{<i!!!>,<,!>ae'a,>,{{},<!>,<!!!!<}!!"!>,<!>},<!>'"!!!>!!!>!>},<e>}}}},{<!!!>,<!>},<!!ua<!}!>,<'"!!!>!>},<u>,<!>!<<>}},{{{},{{{<!!!><,e!!{!>,<i{!>},<"e{!!ae>}}},{{<!u!,{'!!{<!!!>",!>,<"aaai!!!>!}e>},{<'!!!>,<e<!!'!!u!>}a'o!>,<i!>,<!"!"!!<>,<!!>}}},{{<!!i{!}<!>,<'!!!!>},{<'!>},<u"!!e!!!>"i!!!>,<!>,<"u<>}},{<!!!>,<!>,<a'!!!!!>>,<!>},<>}}},{{{{{{<!>!>{<'!>,<<!!<"u>}},{{<}<!>},<!!!>,<",u{!!!!!!{{e}!'>},{<'}a!!i!!!>!!!>},<!!!!!!,uo"e!>!!!>>}},{{},{}}},{{<!!!!u!}!!!>ueo{<eie!!!!iu!>!!!>>},{<ee!,!!!>!!,!!!>ei"u!!!>!>}a}a!o>}}},{{{{<<ie>}},{<oe!!!>>,<,{!!u{>}}}},{},{{{<u,<!>!>!!{{!!e!!!>!>}!>!!!>,<aui<!u>,{<{'<!>},<'}<a<"a>}},<{a!"a}}!}!>!>!!"}>},{{{{<i!e!<uu!>},<e!!!>,<!>},<,<!{<a!>,!>,<<>},{<i<'"!!!>!!,i!!!>u>}}},{{<,!!}o!>},<<{!!!>!!!"a>}},{{{<}ui"!>,<<oo!!!>!>,<!!!!!>!>},<>},<u!>,,!}!!!>!i>}}}}}},{{{{{},{<!!e!!'!!!>'!!!ii!!{!!!>i!!!>,<>}},{<!!>},{{},{<,<u{!!!e!!!><!!''!>,<,!>},<ao,!>},<!!!>>,{<e'!>,<>}}}},{{<"<!!!>'!!!{}!>,<'}!!'<!!!>,<>},{{<,'>},<u'!!!>,!>},<}!!ea!!{>}},{{{<},",e,u>},{<u"ao'!>!!!>{!>>}},{{{{<'!!!!a<u<!>,<}!>>,{<{!!e}ui!!!>!!!>}>}},{{{<{u!>,<u!!e!>},<'!{!>,<u!!!!!>!!!>}e'au<>}}}},{},{{<a{<e!>,<'i!uo!>},<ue!u'!!!>!!io>}}},{{{{}}},{{}},{{<!>i!>!'}ei<"'!}i!>},<!>e!!!!!>},<!o>},<!!!e}>}},{<!!!>''"!>,<i<i!!!>,!,>,<!>,<!!!>!'u"!!,o!!{a!>,<!!i!!!!,"a''>}}}},{{{<!!!>a'ao!>},<!>,<!!!>e!!a!>>,{}},{<'a!>,<!!,<>,{}},{<u!!!>},<!!o!!{o!>},<{!e!!{}!>},<!>,<!>>,{}}},{{<ia<e!'>,{}},{<!>>,{}},{{<'uuu,u!>},<a!!!>!!!>'i!!!>>},<!>},<u!>,<!>},<"u{'!>,<>}},{{{{<!!!>!!!>}''!!!>!>,<>},<!}u!>},<>},<<!>,<!>!>},<!!!!!!'">},{{<u!>},<,ao!!!>,u">},<,o}ui}!!!!!!!>>}}},{{{{<!}a!!!!i"u,o!}>,{<o!!!!!>a!!{!>,<!!u,"!!!!}!!>}},{},{<!!ae!!!{}{!!!!!>},<o!>},<!!!!,!!!>!u"!!!!>}},{{{{<!>}a!>},<i'!!!>,!!!!!>!!!>,<ia!!!"o!!!>},<{}>}}}},{{{<a!aua>,{{<o<{i!>},<!>},<!a,,!!!>!!!>!>},<!"!!o!!!>!>},<}>}}},<e!>,<"!>,<a<,o''!!!>!!!>!!i,o!>},<!!{a>},{<<o{'"eo!>,<<ee,,aue>},{<>,{<!!!>},<!>,<}!!!!!>i{!!!>!>},<!>},<a>}}}},{{{{<!!}!>},<!!au!>},<>},{<ae!!{i'!!<i!>},<,<!!!>!u,!>,<>}},{<,<{!!i!>},<'>,{<}iiuo>}}},{{{<!>,<!>'}!!"!>,<"!>a!>},<u!>!,e!!"!!>},{<o"!>},<!>},<!e,>,{}}},{<!>},<!!!>>},{<ie!>,<!{!>!!!!!!!><"'!>aa"i!>'>,{{<o<!!",!!u>}}}},{{{<!!,{>},<"a}!!!>"!"!!!>o!>!,o!!!>e>},{{}}},{{{{<!>>}},{{<!>},<!>ai!!!>!eu>}}},{<!><e}oa,}o'a!!'!!!>},<!'!!<"">,{<<e!>e{,a"!!!>!!!uiu!>,<!>,<a">}},{<!!!{!!o!>!!!>},<uo!<!>},<o!>>}}}},{{{{{}}},{{{<!!!!}<!>,<u}!><>}}}},{{{}}},{}}},{{{{<"!!!!{!>},<!o!!>},{{{{<,{o!!!>!>,<{{>}},<!!ii<}!oi!>},<<"'!!<!!!>!>,<e!>,<!!a!a'>},{{{{<>},{}},<!}u{!!<a!<ei!!a}!>,<o>},{<{}>,<"{e!!!>a>},{{<!!,,io!>},<!!!e!"!a!!!>"!!!>!!,!i"!!!>>}}}}},{{{<{!>,<,a!!!>!>},<!>!!e>,{{<!!!>},<!!o'!!!>!>>}}},<"}e!!!>a!!!>!!!>!!"!i!!!>},<>},{<!!!>}i>,{<<"',!>!>,<!!!>,!!}i!!"a!!<!>,<>}},{{{<<a'!>},<a'!>'!o"o!><!!oiu!>,<!e!!!>>}},<!>,<e!>},<!u>}},{{},{<}!>,<!!!>'o"!!<e}!>,<!!!>!>,<!>},<"}>}},{{{{{{<,!>},<{"!!!!"}!!>},{<,!>},<i<!!!{{!>{!,!>},<o!>,<<'>,{<>,{}}}},{<!!!>!!!!<!''"{!>,<i"!>,<!>"!!!>,<a!>},<">},{{<'>}}},{{{<a!>},<<,!!>},{<<}!!!>!>},<i!>,<i!!!"u!!!!e,!>},<<u,>}},{{{},{}},{},{{{<!eo',!!!>'e>}},{{<"'ao<<'i{!>>}}}}},{{{{<i!">,{{<!!"!>!>},<!>},<<<aiu!!!>u<'e!e>,{}}}}},{<aai!!<}<!!o'e!>},<>}},{{<o>},{<!!!><{!>,<!!!>!!!!!>>}},{{{{},{{<uiei!o!!o!!u}<e!>>}},{{},{}}},{<!>!''!>,<>,{{}}},{<i!!!>i!"!!uua'!>e>}},{<<!!!>{!!o!>},<,!!!>,e!!,!>},<!>},<}!>},<>,<!>},<u,i{!!!>>}}}},{},{{{},{{<e!!oi!>{>},{<'o!o>}}},{{{<ia>},{<!>!!>}}}},{{<>,<a'!>},<""{!,e!!!><!!'!>!!!!a'!!!>>},{{},{{<"'!!!,>},{}}},{{{{<!>},<ii!>,<aue!>!,!!!>'a<},>,{<!>},<"oo}!!!>"a!!a}!,u>}},<ue{"a}a<}>},{<ia<!!"i!!!!!>!!!!!>},<!>},<<ue<!>i>}},<""'>}}},{{{{{{},<"ee!e!!!!!>!u!!{}e!>!',"!!!>a>},{<"!!!>},<!>!>},<>}},{{<!"}>}},{}}},{{{<'!>},<!!a!>},<'!!!!!!!!!u,!!"!>!!}'>},{<ae!!!>o!i}!!!>>}},{{<oie!!!>!>},<euuoou!>uu''>},<i}>}}},{{{{{{<!!"{>,{{{<<"!!!>!!<!!"!!i,<>},{{},{<}!>,<!!!>u!>,<,!>,<o!>,<!>,<i!>,<o!>},<!><o!!e>}}},<ao!!!!!>e!!!>e!>,<!o!>},<a!!!>>}},{{},{}}},{<u,,>,<!>,<e!'!>,<>},{<'!u"ie<>,{<uu,!!<o!!'!u{!!u!!oo!!!>}!!<"!a>}}},{{},{<e'!!!>>}},{{<}!>u!!!>,<!!u<'!!!>>}}},{{<e,!!!>!!!>a<{!!,{!>!>},<!!!!,!>},<!>!>,<o!!!>>},<aa{!!a>},{{},<'!!e!!!>!>},<!<!>,<!>},<ooi>}}}}}},{{{{<!>!!!>,<!><i"u">,{{},<{ioai!>,<!!'!'!>,<<'!>},<e}!>},<u>}},{},{{{}},{<!!!!!>!>,<"!oa!>>}}},{{{{{},{<e'!>,<!>,<!!!>!!>}}},{{{<>}},{{{<>}}}}}},{{{<!>},<!!u!>iea!>,<o!>!!o>},{{<!!!>!!!>!!"!>,<}ou!!a!!!!!>},<','!!!>},<{>},{{},{<o!>"}!>,<!>,<{i>}},{<u!>},<!e!!,!>},<o!!!>{!!!>{,!>,<i,>,<!!<u!!!>!!!!,!>,<,i">}},{<!!!>}ii!>o""!!!>>,{{{<!>},<>},{}},{<!!!>!!!>!>!>,<ae!}!>},<!!!>!,{}!!oa!!!>},<!!!>!!!>},<>}}}},{{{{<u!!>},<!!a!>},<!!{!!!>,<}!!'!>!>},<'"e">},{}},{{},{{<o,a{<e<!!!>!!!<!>},<{>}}},{<}u!>},<"<a!>,<ooa!!!!u,>,{}}},{{{<}e!>},<>},{{<'!>},<!!!>a!>'!!!!}o}!<{"!!>}},{}},{{{}},{<!!{{<i!<ou!!,!!!!!!!!>}},{{{<!>,<'eu!!!>'!i'<!!a!>,<"'!,,,!>},<!!<>},{{<"a{!>},<!!eu!!!>}!!!>},<!>"!!!>!>,<,!eo!u>},{}}},{{<!!!!!e<!!!i!>ie"!<'!!!>>,{<!!!><>,{}}},{<!!!>{!!!>oui}{,'!!!>o!!}"!>'>,{<}!!!>},<i},<ie!!!>i!!i>}},{{<<u!>,<<u!!!>,<!>,<{>,<'<<"a!>},<!!<!>,<!!!><u!!!>!>!!!>,>},{<"!{!!o!>,<<<!>e>,<!>},<e!o'!!!!{}"!!a!!!"'ie<!!!>o>}}}}},{{{{<eeea{!>ua>}},{<o!!!!!>!o!!<!>!!!>a{o}<}o!!"uiu{>}},{{<!a}e}"!!o">},{}},{{}}}}},{{{{},{{{<!!!!}'{!!!>'<>},<!!<i<,!!!!!>},<!>!>},<"""'!"!>},<!!!>}<!{>},<!>},<!>,<!!!>},<!!!>eie{!!i,!!a!>!!!>!}!!!>>},{{<e,}i!!',!"!>"i!!!>o!>},<,<!>},<>},{}}},{{<!>,<>},<<iua",<!{>}},{{<!!!>}',!>"!>},<!,!>,<!!!>!!'!!!>!o!!!>>},{{},{{{{<!!!>!o}ii!!'!!!,o!!!>u!!ee<"i,!!!>>}}},{{<!!u'<a!!'!!,i!,>},{<!!o!>!>!!!>},<<}uo!!!>'>}},{{},{{},{{{<>},<!,!>,<>}}}}},{{{<<!>},<!!!>!!!>e{>},<!>,<<!!!>oee"!!!!{<>}}}},{{<o'iu!!!>u"}}!!{>,{<u!e<e!>,<!!!>!!!>},<!!!!!!>}},{<!!<<!>!!u!!!>i"!!oua>,{{<<!!}!!}!>,<">}}},{<}ui!>,<}"!!!!!>!!!>>,<!iu!!!!!>a!>>}}},{{{{{{<}!>,<}!>!!!>'"!!!!!>a!>,<!!!>e!>,<{>,<!>,<!!!>'eo!!"!>a{u!!e{a'!e>}}},{},{{{<i!!!!u!'"!>,!!!>{,}<!>,<">},{{<'!>},<!i'!>,<!>},<ui}>},{<u,}>}}},{<>,<{}"<{!>'!u!!!!'}'<!>!i>}}},{{{<{o!u,a>}},{{}},{}},{{<!>{!!!!!>!!!!!>!>,<{{!!>,<i!,!>},<!!e<i>},{{<{ao!!!>""a'!'!>},<!>,<!!}''!!!!!>},<,o!!!>>},{{<a!!!>'<">},<'i!!!!!>u}'!>a,"!!!!!>,<!!!!!!i>}},{{<!>},<{!}>}}},{{<!!{!!a!>},<uu},!!!<>,{}},{{<{!!!>!!"!!!!e{i!>!!!!!>,<o{a>},{{},{}}}}}},{{},{{{<,e!!o}!!!!!>,<!"i!>},<o}>}}}},{{{<!>>,{<!!!!<!>},<'{a!>!uo!,!!>}},{{<'i!!!>i,>}},{{{}},{<!!!>},<!!!>!>},<,o!!i,}">}}},{{{{<{!!!!,"!!!!,ee}<,o!!!>!{!>},<,!>,<!!<{>,<!!}!>},<o!>!u}!!!>!>,<!!!>i"o!!ii!!!!aui>},<!>e<!!!>!>a!>},<!>},<!!!>!!,i>},{<!!,!>},<>}},{{<aoa>},<,!>,<!>!>,<!>},<}!>},<!!!!!!a!!i!!'{u!!!>!>,<!>,<<!!!>!>>}}}}},{{{{{{},{}},<!>{>},{<!>{<a<iu!!"{i>}},{{<'u!>,<,!u>},{<!!!>},<aa!o!>,<"!!!>!>,<!!!>!>}a}e<>,<"!!!>}"e}eo'{e,ai>},{<<'!>,<{ao!<!!!!!>!>,<,!!!'{>,{{<!>},<eio>}}}}},{{{{},<,{!!!>},<{<}">},{},{{{<o>},{<<!!}i},}!>},<,o>}},{<ie!aa}{e"!>,<''"!!}'{u<>,{}}}},{{{}},{{{<!>,<!!{!!!u!>},<!!!!<!>,<>,<!>,<!>},<i}i<e!o!{!>,<!!!>!>>}}},{<!>},<!!!>!}!>o!>,<!>},<o!!{!}!!a{!'eu}i>,<,i!!!>},<euee!!io"!a"i!!!>!>},<i>}},{{{<!<!'!>!{!!!!'<',>},{{<!!'!o>},{{<>}}}},{{<!>>,<'a!>},<"!!u<}!>},<!>!!!>>}},{{{{<!>!!!!i!!o'i!>!>!>!>,<>,{}},<!!!!!>,<!!aoe!>,<!>,<>},<"<!>},<!,<<!>},<<!>!!!>!!>}}},{{{{<i}<e!!}!oe'!!!!a>},<oui!>}>},<!>},<'!>},<!!i!!eu"!!!>',o{>},{{<!e{e}!!!>i!a<"!>u!!!!,,!!!>e"!!!>>}},{<{}!>,<!>},<i<a!!"u}!!!>!>,<!>},<!>,<'>,<!!!!!>!>>}}},{{{{<!{}!>,<!>,<>},{{{},{{}}}},{{<o}!>"!>},<!>,<,a}<!a!",ae{>,<!!!!,a,<,'uoo!u!>},<!o!!!!{!u{,""!>,<>},{<}e!!!>,<}"{}>}}},{{{},<!!ei>},{},{}},{{},{<!>},<!!!>,<uo}>}},{{{{{<!uo!>},<!!!>}eia!>,!>!!!>!!<{,!!!>!>,<!!!>!>,<>},{<!!!!!>!{<!>!!,>}},{{<u!!!,u!>},<>,{<!!!><"a<u>}},<!<!>},<!>,<'e>},{{{<!>},<}>},{{<!>},<'>},<}e>}},<i!!'{!!,e{{}!>e,u,e!!"">}},{{{{<!>{!!!!!>}!}!e!!!>!!"'{'!!!!!>>},{<!!!!!>>}},{{{}},{}}},{{{{},{<io,<<>}}},{<"<<!!ie!>,<o<!>,<>,{<"!>,<!!!>,<o!>},<{!>aai!>,<i!!!>>}},{{},{<!>},<u{e!>,<!>,<!>,<!>i!!!>!}!!!>{i!>,<!>},<ie!!!>>,{<'!!!>}!"e!!!>u!!!><!>},<!!ee>}}}},{{<>,{}},{<"a!>},<}}!>,<!!'!>},<!>>},{{<!>,<!!!>!!!!!a{!>},<!>,<!,!>},<!!!!<io!!!>'!>,<e"!!'o>}}}},{{<,'u>},{{<!!!>!}"!"!}!>u!!o>},{{},{<!,'!!!>,<!>},<e!<!>!!{!i!'}}!!'>}}},{{<}"!>,<!>{!'!>,<!>},<i!>"}!><<o>,<!>!!!{!>},<!<!>},<,a!>!>,<!!e'!!o,<">},{}}}},{{{<!'!!!>},<i>},{},{{{<o<uauiii"ea!!""u!!eu>},<"'">},{{<,e!!!>'<>}},{{<a!!!>,<'}<!!i!e!ae!!!<"!!!>u!>,<!>},<!>},<!>,<i>},{<,,o!!'!>>}}}},{{{<!!!!,"i!o!"ao}a>},<,>},{{<{!>{>},<!!!!!>"{!!!!{!!!>!>,<io>},{{<>}}},{{{{<iee}!!"!!!>!>,<aee"}>},<!!}ue!!e!>!>'e!>!!!>ue!!<!!<">},{{{}},{{}}},{{<e!u'<e!!!>e!>!!!>e!>},<!!!!!>},<!!!>'"!>,<!>,<<!!!>},<>,{{}}},<!>},<}!>!>'<!>,<!>},<!!!!!>e!!i!!!>a!>,<}'>}},{{}}},{{{},{<!!!>,<'!!,<'e!!'!!!>!!<"ei!!!>!'{<>,<o'e!!!!!!a!!ea!>!>'>},{<{!>a!>,<e!>},<e,}uioe}ao!>},<!>!!'>,{}}},{{<!!!>!!!>!!!>!!!>>},<!>,<a!!e!!!!!>,<!>!>},<!!!!a!!u!>,<"a>}}},{{{{},{<>,{}}},{{<'"}i!>,<,},<{}!>},<u"e{>,<!!!>!>},<i'}!>o{>},{<!!<!ae"e>}},{{<!>!!!>!!"<"!>u<}ua>,<'!!ou!>},<e,>}}},{{{<a{"<"!!!>},<oe!>'ie}"u>},<",ii!!!>!u{o!>},<io!>},<}!!!>!!!!!>>},{{{{<,!>,<i"<u!!!>},<oeiua}'!>},<u}>},{<>}},{{}}},{{<"a,}u!!o<!>},<!>>},{<!}{,}!!!>>}}},{{<}e<!}u!>,<e!!'ao!!<!'!>>}}},{},{{<,'eee!!!>e>},{{{<!e!>},<<ui}u!!!!!>uo}u"!!!>u>},{}}},{{<i,',!!,uua{ii!{u>},<ai!!!>!>,<!!!>!!o!!>}}}}},{{{<{}<!!<!!!>,<{"}!!!>!!!>>},{{{<e!!e!!!!i'!>},<!>!>u!!!!{ei>},{<aoao!>},<!>!>,<a!>},<!!!>}!!>}},{<!!!!!>},<!!e!>,<!aa!<!>},<!!!>!!>,{}}},{{{<'e!<i!!ao!!e,!!!e!>,<!!!{eo!>,<{>}},{{{},{}}},{{{<e!!a,!!!!'!o!>e!>'u>},<"o!'a!>,!>},<>},<a,,>}}},{{<!i!>'!iu'!!!>},<}!>},<!>},<u!!!u!!<,!>,<!!!>'a>},{{<!!,!,>}}},{{{{{{<!!!>{!>},<{}!>o,}{o">},{<'}!!o!!,!!!!!>"!>,<<!uo!ioe!!!>!>,<,!"{>,{{<!!e!!!>o,o!!>},<'o!>!>,<!!!>,<!"<e<">}}},<ao{<!!,"<!!!>u!!!!oa{a>},{<!}!!!uu>}},{<!!!>!!ii!!!>{!a!!o>}},<!>,<a"}!>},<ui"'!!{!>},<!!!!o>}},{{{<!,!>,<!>,<"!!!}!!!>,<i!!i!!<!!!>!!!"a",!!!>!>,<>}},{{{{<<u!a<e!!}>}}},{<!!!>">}},{{{<e"{!>},<!!io<!>,<!>!uoe'e!>},<!>},<!>},<'!>},<>}},{<}{ea!'"au!>},<e!>,<!!ieuu"ee!!!>>}}}},{{{{{<!"{"{!!!>i!,a>},{<<"u!!o!>},<"{!>},<!!!>{{e>,{{<!!a!>},<!>,<!>"!>,<!>},<u",,<>}}}},{<!"oi!!",!!!!!>ieii>},{{<!>},<<<}>,{}},{<,o!>!!{!>},<!o,,e"!i<"o<<e'>}}},{{{},<{!>,<i!!{!>e!>,<e!',}>}}},{{},{{<"!!e}i}e!>!>},<<!!a!'o!iui!!!!!>e>},<'!!!>!>,<}",!>,<<!e!>},<a!!!>'u<!>,<e>},{{<""!!}"!!!>{<>},<{i}!>},<a{!!ue}!>}{!!!>i!!!>o>}},{{{<u'!>,<<!!!!"!>,<"<!!!!!>},<!>>},<e!{'!>},<!!!>!>a"a!,">}}},{{{{<a!!!!!>},<{!>!>},<!a!!!>},<>},<!!!!'a<"<a!>},<!>{,<!{{!>},<!!e!>{>},{{{{<{ea!>},<u!!!>>},{<<!>{i!,'!!'!>,<}!>},<'!>},<!!,i!!a!>,<>}}},{{{<"!>},<"!>,"!!!!!<}"!!!>!{,>}},{{}}},{<,oe!!!>!!!>!>},<uei!!'>}}}}}}'''))

12505

## Part Two

Now, you're ready to remove the garbage.

To prove you've removed it, you need to count all of the characters within the garbage. The leading and trailing `<` and `>` don't count, nor do any canceled characters or the ! doing the canceling.

- `<>`, `0` characters.
- `<random characters>`, `17` characters.
- `<<<<>`, `3` characters.
- `<{!>}>`, `2` characters.
- `<!!>`, `0` characters.
- `<!!!>>`, `0` characters.
- `<{o"i!a,<{i<a>`, `10` characters.


In [23]:
def count_garbage(input):
    canceled = re.sub(r'!.', '', input)
    count = 0
    for m in re.finditer(r'<([^>]*)>', canceled):
        count += len(m.group(1))
    return count


In [24]:
[count_garbage(example) for example in [
    '<>',
    '<random characters>',
    '<<<<>',
    '<{!>}>',
    '<!!>',
    '<!!!>>',
    '<{o"i!a,<{i<a>'
]]

[0, 17, 3, 2, 0, 0, 10]

In [25]:
count_garbage('''{{{{{{{{{<}>,{<<'!!i}!>!!!!!>},<{,'!>,<!!e!!!>!!<a>}},{{{{<"o>}},{<!>,<!!!!o!ei"!}>}}},{}},{<},!!ua!!!{{!>,<!!!>},<o>}},{{{<o!>,<!!!>!>,>}}}},{{<,>,<>},{<'ui}<>,{{<!!!>,<,!,!>},<'}>,{<<!!}<<!!!!e!!!>}!!e>}},<e!!!>,!!!>ao'!"!!ooa!>},<"!!!>,<>}}},{{<i!>!>,<!>,<!u>},{{<oe!!<ii}i>,{<!>,<o}{{}a}!>},<!>},<!!!>!{!!!>!>},<}'u!o>}}},{{{<}!!!!!>},<!!e!>eu!>},<e>,<o!>,<!!!>"i!ao!>'}"eia,!>},<>},{}},{<<}!!!!a'!!!!!!a!!>}}}},{{{<i'!!!>!!!>!!!!i""!!u!!>}},{<!!!!o>,{}},{<!!"!<o"!>"{!oo!>,<!a>}},{{{<!!!!!<!!ou}>},<ee'!eee!!!>},<>},{},{<u{!>e!!au!!o!>},<!!!!!><'!!io'>}},{{{}},{{{<!!!!i,!!{}>}}}}},{{{{}},{{{{<<!!e{i"{,>},<!>},<i!!o<!!a'!!!!<e"a>},<!>e!!!>},<!a!!>}}},{{<!i}!>},<!!{i>,{}}}}}},{{{{{<!!iai!>},<!>},<,a!>>}}},{},{{{{{{{<!>},<a}<i{,{{!>},<u>}},<!>},<!!!>,<ua{!>},<!>},<!!{e}'ui>}},{{},{<u>}}}},{{{{<!!!!e!>,<}<e!>,<!!!!{i!a!!!>o"!!!>{>}},{{<ao!>},<e>},<!!!>,<!!!>e,!>!>},<u!!a,""{!!!>,<{>},{{{<!>},<'!!,!!'e"o!>},<!{,!>},<'u!"!"o'!!'>},{<!,!!!>!>},<!!<<!!!>},<{!>,<e!e{}!>},<!!<>,<i{!>{!>e"i!!!>{u"e!>},<a!>,<!>},<i>}},<!a!!!>!oo>}},{<<{i>,{{},{}}},{{{<!>,<<!>},<,,o{!>,<'!>},<u{!{}u!}!!!>},<>},{{{<!>!!!>},<}",!!<!!'>},<a!!!!!>}!e!!e!<u!!,''aa{!!o}'u>},<"'!!!><e<"!!<e!>,<i,'>}},<{!>},<e!!!>!>},<!o!!!!!>">}},{{},{<o!!ui!eia}!>,<oai>}},{{{{{{},{<'<<ao!o!>,<>}},<!!!!!>e!>!>},<!!!>!!'>},{<!!i'<!>,<!>!!}!>},<!!!>>,<!>},<!>},<!>,<<!>},<!!<!>'!>o!!u'!!!>i!>o,>}}}}},{{{{<o!!!>",a!,{!>},<',!!!>a!>"}!!!>},<>},{<a'!!!>,<<<<!>,<!!!!u!>,<!!{>}}}}},{{{},{<!u<!>},<>}},{{{{<uu!!!ai>,{<u"!>!!!>!!'i!e!!!>,<>,<!u'e!}oi,i}u<!>},<"}o"!!"'>}},{<!>,<eo}'<!!!!!>>,{}}},{{<>,{}},<o}u{e!!!!!>!>},<!!!>,<>}},{{{{<!!!>!>,<!>a,i'!>!,i!oeu!!!>,<!!!!!}"!>,<>},{<!{a,{!>,<{!>!!e'!,u!a!>!!!>o>,<!!!>!!!>o"<>}},{{<!u!!!>!>!!!!!>},<a!>a>}},{{{}},{{}}}},{{{{<"<o!>},<o'o!}{i!>}}>},<!"!>,<e!>!!!>!!iui!>},<,!!"a>},<!>},<u!!o!!!>!!'!,!!e!>}a!!"a!!e>},{{{}},{{<}o'!!'!"o!!au>},{{{<a!!u"!>'!!o!!!>""}'iui>}},{<!i'o{!!{!!!>!!}<!>,<o>}}}}},{{{<>},<o,iae!>,<{{<'<!>},<{!"!>},<!{>}}},{{{{<>}},{{{{},{<'!>!!ii!>},<!"o"a!>u>}},{{},{<,!>,<!>},<{{e'!!!!!>a'e"ei{!u!>,<!>},<!>>}}}}},{{}},{}},{{{<,u!!!>,<!!!>{<!!eo!!!>,<{!!",!ee!!!!>}},{{<e!!!>u!>,<e>},{<i,<!>},<ou!!>}},{{<!>,<'}<"!!'>},{<!!}!!!><!!!>'ea!!'!,!!!>'e!>},<"aae>,{<!>},<!ii'e"{a}o!>'!!o}"}>}}}}}},{{{<!>,<!!!>,<!!!><'!>e!!!>">,{<!!'!>,<{>,<,!a!>,<i<!!}"!!'u!!!>,<>}},{{},{<!>},<!>},<{,{!!!!""{!>,<!!,!!!>u'>}}},{{<!!!>,!!{!>,<<e{i{>}},{{{}},{{<!!!>}u<}!}!>},<!>},<<!>,<i}e>}},{<{'!!o}!i!>,<i!!u>,<}!>},<!!!>,<{!>},<a!ai'"!>,<oe}o!!!>o!>},<!!!>>}}}},{{{{{<}i!!a!>>},{<!>e!e}"!>>}},{{{{{<},!>},<au!>a>},{<,u"!!!a!ui>}},<{}'<oo!u!>a!>,!>},<u!!o'}!!!>>}},{<!>,<{!!!>},<!!!>!!!!!>,<"e!!!}"!!!!i!!!>,!!a,i!!!!!>},<>},{}},{<!!!>!>,<!o!!!!!!!!,!>},<'!>,<}>,{{<i!!'i,!>,<{'!!o!>,<!!!!"ii,!>i{">}}}},{{{{<i{>},<"u!,'">}},{{{{}},<!>,<!i"a>},{{},<o"!!!>!>},<!!!>>},{{<i!>},<o!!!>!>},<!!!>!>},<uo!!!!!!'aa!!!>e{!!a,o>}}},{},{{{{<<!},{!!!>a'}i!>},<!>},<<"!>},<!e!e!'>}},{<aiu'!>,<!"{!!o!!{!>,<,}>,<!o"!!<!">},{{<!!u!!!>},<!!!>!>},<au!!!>!>},<}!!"'i!>,<<i!>},<>}}},{{<"!>,!o!!>,{<!>,<e>}}}}},{{{{<'}<!>,<!!>}}}},{{{<!!iuo!!!>!>},<!'!!!>},<>,{<{!>},<!!"!<>}},{},{<>}},{{},{<"o!!!>!>,<'<'ua<}"!!!>,<!!!>!!{!!!>>}},{{{<!{!>},<!!!>i!>},<!!<>}},<'eui!ii!!o>}}},{{{{<!{!!!!!o>},{<eu!uuu}!!,!>},<,e{>,<!!}<!!!>!!e!!!>!!!au,!>,<{oio<>},{{<}e,'!>,<!!,!!!!}},!!!!!!!!!>!aou>,<e<a!a'!!!!!>o!}!>},<>}}},{{},{{<!!!>!!!>!>e}uie>}}},{{<e!!"!>,<!{{!!!>oi!>,<,!u>},{{<!!!>>},{<!>!>!!!>!>},<{!iee<a!'!!!!!><!!!!"!>},<'a>}}}},{{{{{<!>!!!>{{'oe<}a{}>,{<}!>},<!!!!!><!>,<o!!!>!!{o!e!>>}},{<a{,a!!}"{!!!!!!!!!!!!'i!!"!{<i,!>,<!u!>},<>,<{,!>,<!>!>},<!!!!i!!!>!!<e>}},{{<!!!>{!!>,{<o!!>}}},{{{<!>"o!!!>!,!!{!'"!>e!!!!u!!!!!!!a!>,<"!>},<>,<a"!!!>!>,<<,e!!!!{i>},{{{{},<<!!!><!!>},{{<!>,<o!>},<!,i!!!>},<<!>}!>,<!!!>>},{{<}i!!!>!!<{}!oe!!!>ie"!a{ia!>,<>},<!!!!oe!!!!}<!>},<o!!u"!!a>}}}},{{{{{},{<!<'!>eu!>,<e{'i}!!!>>,{{}}}},{<!>!>},<!"!!!>>},{{{<!!!!"u"!!!>!!!!!>!'!>,<>,{<!>},<!>},<!>},<!!!>!!!>!>!!!!u!!i!!<!!!>,!!{!>},<i>}},<'i'!a!o!!"!!,!!u{!{o{i'!!u>}}}}}}},{}},{{{{{<<!!!>}eu>}}},{{{},<'aa!!!>!>'}u>},{<}o!!'!,'<<<!!!<!>},<u}>,<!>},<"!!!>},<>},{{{<!>,<!{u!>},<>},<!!a!!!>o{}!!"'!e,'!!!!!>,"!>,<i{>},{<!!<!>},<,!!<!>},<>}}},{{<au!!!>!>,<oo!>,<a}u{'u!>},<!!'e!!a!>,<>,{<o!!oo"!!aua,!!!!eoiaea>}},{{{<ou!>,<oo}!>}!!!!!!{"}i!!!>,!ou{<>,<{u!>},<!>,<<!>!!!>},<!e!}<>}}},{{<!>!>,<i}a!!,}!!{!!!i!!u!>>,<}o!!>}}}},{{{<!!!>,<">},<!>,<''a'!!a!>!!!!a!>ia"!"!!!'{!>,<!>},<>},{<!}!!!>>,{<!!e>}},{{{{{{<e!>,<!!!>},<e!e!o!!',a!!!{'u{!e>}},{<o!>},<>},{{{<!>,<!>,<<!!!!!>o!>},<"!>,<io!>},<eoo,e{!i!u>},{{},{{<a,uu>}}}},{},{}}},{}}},{{{<i,>},{{}},{{<o!>},<u>},{{{{{},<,<o"}!!>}}},{}}}}}}},{{{<"!>},<!>,<!>},<ea!>},<!!{!!!!'i!>},<"!>,<""a>}},{<uo!!}eo!iua!i">,{{}}},{<!!!>a{!!!>,<<uo!>,<,e!>},<!"'a}>,<!>},<i!}o>}},{{{},{{<u"!>},<!>!!!!a{!>},<!>,<>}},{<!!',!!!!!a!>,<>,<'>}},{{{{<eia<<ua>},{{<>},{<!>,<!>},<!>},<{"i!>}">,{<!'!!"!>!>!>,<!!!>e>,{<!!ue{{{<'"ieu{<ee{>}}}}},{{<!{,!!!>!>},<!!!,!>,<!>{!!{>,{{{<!>},<!!u!>},<<!>},<!>},<!!ae<!!!>,<!>},<!'""{o>}}}},<'!>,<}{!!!!}e!>},<i!!!!!>'!>'}!>e>}},{{<"',e}a!!!oe!>},<!>,<!!!!!>!!!>,i{u<>},{<",a'u}<!!!>{!!a,{"'!!!>},<!>},<!>,<!!'>}}},{{}},{{{<},!!!>},<"!>},<'i!!!>u!!!>e!!}e"e>}}}}}},{},{{{{{},<u!!!!o!'}!!!>e'e!!e'!!,e!!!>!!<!!,>},{<!!!>i<!>}!!}}i<<!!,!!{>,{<>}},{{<i,a!!!,>}}},{{<!!!!,"o!},!!!>},<!!{e!ioo!>,<<u}!!'>},{<!>a!!!>"!>,<!!!>,<e!>{u{!!{!,<!!'>},{{},{<!>!>},<!!!>},<',,!!!!!>},<u,e!''!!!>}>},{{{<<<{!>,!>},<'{!!!>!>,<!!}!!{!!!!!>!>},<"}!!!!!>,<>},<!!!!>},{<{ae!!!>!!!}!!!>"!!!>},<a!!!>o}!>},<<!!!>>,{}}}}},{{{<o{'!>},<e"o!>!!uu!<e"i>}}}},{{{},{{{{},{{<u>},<{!><eea!!!!eue>},{<i!>!!!>},<!>,<!!>,{{{<!!!>!>,>},<}!>,!!!!,}e>}}}},{{{}},{},{{<!>},<!!!>"!!o!>,<{au>,<!!{"!{!">},<},e!!!!!!!>,<!!!'!!"!!{{'>}},{{{{{<!!{'!><'!!e!!!>},<>}}},{}},{{<{}!!ie<!>!>,<<u"!!!o>}},{{{<"!>,<u!!!>},<>},<!!ea'!>},<"a!>,<{!!!>},<!>},<!}!'!"!>>},{{},{<'!>!!!{!!,"e!>},<!!!>'>}},{{},{{<'ua!!<'u!!a}<}{'o>}}}}},{}}}},{{{{<!!!>,i!>!!!>,<!>a!>>},{<eo}a!>!i!>,<>}}},{{<!<{e,<{a'<i!>,<!>!!e!!!>aio!!<>,{<!,u<}!>},<}!!!>!>!!'!!}!>,<">}}},{{{<a{}!!!>'ia<!>},<e!>},<>}},<!>}"a!!!>!!>}},{{{<a!!{!!!>>},{<<!>},<,,<>}},{{}},{{{<i!!{i!>!i!!!>>}},{<!!{e!!!>}!>!!!!!!',!>,!!!>"a!!!>a"!>,<>},{<>,{{<{'>},<!!a""{!!!>!>,<{!>,<}">}}}},{{{<!!!>!!!"u}ae"'i!!}>,<!i!>a!!!>!!!!,!>i>},{{}},{{}}},{{{}},{{}},{<!e!>},<!>},<!>,<e<!!'>,{<,"!!,!!!>!>'!!!>'>}}},{{<!!a,o!u,o{o{o}}e>},<!>,<,,o"{e"!i'!!!!!>,<!!u!!!!!><>}}},{{{{{{}}},<!>,<!!!>,}u!!{!!!>},<!"{!'<!>,<!i>},{{<!!!>e!!!>u!!o>},{{<i,>}}},{<a!>,<{!>!>},<{!!u<<>,{{{<!>!!!>,<!o}!>,<"!}>}},<!!!>!>},<!!!,''!!au}!!!>!>!>,<">}}},{{},{{<!!!>u!!ea>}}},{{{<{,}'}!!e!>!!,'<!!!!'"!o>},{<!>,<!>,<a!>,!>!!!i!>>,<eoo!i}!>,<}"<e!>!>},<o}!>,<}!>e>},{<!>,<u!!!>},<"!>!!!>},<>,{<!>ao>}}},{{{},{{{{<>}},{},{}},{{<!>},<a!!!>!>,<,ea!!aa{i!>"<>},<!>,<a'ue<!!!>},<>},{{},{{},{{<!!!!"u!!o<'i"!!ou<i>},{<"!!!>"i!!'!!,u!>},<a'!}}a>}}}}},{<o!!!!!!u!!'!>,<!!!!a!u!>},<'a,!!!!!>o!!!>{!>,<!!i>}},{{{{{<<>},{{<!!!>!!'!!!>'i!!{!>!>,<,u>}},{}},{{<o!!!>}i'>,{<'!'e{i,"<!>,<}>}},{<!!e!!,e{i!!i!>},<i!!i!<!>},<!>},<e!!!><!!">,{<!!,e>}},{<}"!>>}},{{{{<!>,<a}!!o"eae<!!,!<!>,<'a>},{<}"'u!!!>,<a!>},<e>}},<!!{{!!o,,!!i!>},<ii!<'<}'i{!!>}},{{{<!!!>u}!!!>!!!!}'}aa!!u!!!>>}},{},{{<"!!>,<a>},{{<'!!!!!u{!><!!!>!>,<,">},{<!>!>!>,<>}}}}},{{<i!>},<i,"!!!>>,<!e,{i,i!o<!>},<a!{e!>,<>}},{{{<!>,<<<>}},{{},{<'!>},<!!!>},<{!>,<}!!a}oe!a'o!{>}},{<!!o}!>!>>}},{{{<!>,<!>,<a",!>,<a!>!!!>>,{<!>,<!>,<i!!!aau}!>},<e<!>,<>,{}}}},{{{<!>},<!!a!!,!>"i!!!!!!!>,!e!>,<o'!!!!!>"<>},{<!>},<!!,!>},<!>,<"!!!>},<o>}},<}'!ao!>e}>},{{<!!!>i!>{!>,<>}}}},{{<!>,<'}{"<!>,<'!!!>,<!>,<>,<!>u}!!"!aa!>,<{!>!"!>},<a!>},<!!!>,<>},{{{}},<!!<!!!>!"!!!!"!!e>}}}},{{<a}!!!!!!!>!<>},{{{<!>},<ee!>a<>},{{<<i!'"ua!>},<!>},<,!!!>!>,<,!>},<>},{{<!!!!i!>{!!!i!!!!!>!'a!!!>o!!{oo!>},<>}}}},{{<!>}!>,!!!><,i{o"!eauue>}},{<"!>u!!{<!i"!!}}!>},<!!a{>}},{}}}},{{},{{<iu"{!>,<!!{!>!{o!!"}!!!!'!e{>,{<!!!!!!!>!!!>,<!!!>"!!'!!""!!!!!>,!>,<,!>},<',}!!'!>,<>}},{{{{<!!'>,<>},{{<ou<ua>,<!,!!>},{<o}o!!a"}io>}},{{{{<aeo!,!>,<}'!!!>ei!>,<<!>,<oa>}},<eo<ie"!>},<!uu'!!!><,}"!>},<>}}},{<i!!!>!!!>ia<iu!!{!>},<>}},{{<u!o!!"iu{!>,<e{!!!>u!>},<>},<!>,<!'}}}!>,<!ue'e!!}>}},{<a}a!!}"i,,!>>}}}},{{{{<!!!>},<>},{<a!>},<<,!e{!>,<<}o!!'i!>,<,!>,<!!!>,>}},{{<<<'!}i!!!>}a!!!!!>!>,<,u,!!!>!!!>>},{{<>},{}}},{{<!>},<<!!!,u,{!!!>}!!'!i!!'>,<ai!!<io!>},<!!u<>},{{{{}},{<}a!!!>}a!!}!!!!!!"a>}},{}}}},{{{{}}},{<!,,!>{!>eeoa!!e>,<!>!!,{"{!<"}a<a!}ei,!a>},{<<!>!{!iu!>,<uaa>,<!!,e>}}}},{{{{},{<"!}!"!>},<!!!!!>,<!aa<!u{"!>{u"">,<iuoe!>},<<!>,<i!!<o!>,<u<e!!!!}>},{{<!!{{!!!>!!!>!>,<!!!>},<!>,<!!!>>,<!au{!!"i'>}}},{{},<"u<iu<a>}},{{<{!>},<i"},{ue!>u!!!>i{!>,<}>,<"i<!>},<u'a>}},{{{{<,}<o!!u!!!>>},{{<!!!><!!,{i{!"!!,'<'ou!u'!!"!!a>}}},{},{<!!!>!!e!!'!!}!><i'!!!>},<!!a!>},<{!!!>u!>,<u>,<!u!>!!}!!!>},<>}},{<i>,{}},{{{<<i!!ea!!!u,}!!!>u{!>!>,<<"u}>}},{{<e{<!!!>iaui,i}ou'{!<}>,{{<i!>!>},<!>,<"<!!!>!!"!,eu,!>,<>}}},{<>,{<<!!!>,<!>},<!>},<'"!!!>!!e>}}},{{<!>},<}!!<'!>},<!!!!!>"!!!!{!>,<,>},<!!!>},<!!!>u!>,<'"!e!!!>!!!>!u>}}}}},{{{<""'a!!!>!!'ue{u"ui'!>,<>,<<ea!!!!!>,<!!}aa,i{,!>,<>},{{<!!!'!>!!!><!!u,o!>>},{}},{{{{{<!>},<<!u,!!!e<i<!}o{>}}}}}}}},{{{{},{<oa"{<!>,<'}!!}e!!!>'>,<!>,<"!!!>,<!>,<e',u!!,,{o!!!>o,>},{{<!!!>a"!!"!!!>},<},i<<,e!>ii<>}}},{{{},{}},{<>,{}},{{<{!!!!}>}}},{{{{},{}},{{{},{{<!!!>,<!!!>},!>,<i!!!!!!!>{!>},<e}>},<!i!!{!{!>!!a!!!,<{!>,<{'e<a',i>},{}}}},{{{{{<!>},<{e"!!<!>},<a{e<>,{<,!!!>!e"o<,"<i"}!>},<uiu}>}},{<a'!!!!!,u!"!!!,}!>e!!!<ui'!>!!{a>}},{{{{<u!!!>!!!>"<"e!>',o!!!>u!>!!!!o!}>},{<!>},<{'eo>}},{{{},{<{i!a<a!!!'!!a!i>}},{{<!>,<{,}!!!>!ee}!uo},e,e!>},<u}>},{<!!!!!>,<"<u<!!!>!!'"ea!!,!!!!!!!>,<<a!!a>}}},{<ou'!>,<>}},{{<!i''o>},{<"}{e!>},<!!!>o!>},<>}},{}},{<!>},<{i{<!"!!!!<eo!!iu!!,!>>,{<e!>>}}},{{<i!!!>,<,!>ae'a,>,{{},<!>,<!!!!<}!!"!>,<!>},<!>'"!!!>!!!>!>},<e>}}}},{<!!!>,<!>},<!!ua<!}!>,<'"!!!>!>},<u>,<!>!<<>}},{{{},{{{<!!!><,e!!{!>,<i{!>},<"e{!!ae>}}},{{<!u!,{'!!{<!!!>",!>,<"aaai!!!>!}e>},{<'!!!>,<e<!!'!!u!>}a'o!>,<i!>,<!"!"!!<>,<!!>}}},{{<!!i{!}<!>,<'!!!!>},{<'!>},<u"!!e!!!>"i!!!>,<!>,<"u<>}},{<!!!>,<!>,<a'!!!!!>>,<!>},<>}}},{{{{{{<!>!>{<'!>,<<!!<"u>}},{{<}<!>},<!!!>,<",u{!!!!!!{{e}!'>},{<'}a!!i!!!>!!!>},<!!!!!!,uo"e!>!!!>>}},{{},{}}},{{<!!!!u!}!!!>ueo{<eie!!!!iu!>!!!>>},{<ee!,!!!>!!,!!!>ei"u!!!>!>}a}a!o>}}},{{{{<<ie>}},{<oe!!!>>,<,{!!u{>}}}},{},{{{<u,<!>!>!!{{!!e!!!>!>}!>!!!>,<aui<!u>,{<{'<!>},<'}<a<"a>}},<{a!"a}}!}!>!>!!"}>},{{{{<i!e!<uu!>},<e!!!>,<!>},<,<!{<a!>,!>,<<>},{<i<'"!!!>!!,i!!!>u>}}},{{<,!!}o!>},<<{!!!>!!!"a>}},{{{<}ui"!>,<<oo!!!>!>,<!!!!!>!>},<>},<u!>,,!}!!!>!i>}}}}}},{{{{{},{<!!e!!'!!!>'!!!ii!!{!!!>i!!!>,<>}},{<!!>},{{},{<,<u{!!!e!!!><!!''!>,<,!>},<ao,!>},<!!!>>,{<e'!>,<>}}}},{{<"<!!!>'!!!{}!>,<'}!!'<!!!>,<>},{{<,'>},<u'!!!>,!>},<}!!ea!!{>}},{{{<},",e,u>},{<u"ao'!>!!!>{!>>}},{{{{<'!!!!a<u<!>,<}!>>,{<{!!e}ui!!!>!!!>}>}},{{{<{u!>,<u!!e!>},<'!{!>,<u!!!!!>!!!>}e'au<>}}}},{},{{<a{<e!>,<'i!uo!>},<ue!u'!!!>!!io>}}},{{{{}}},{{}},{{<!>i!>!'}ei<"'!}i!>},<!>e!!!!!>},<!o>},<!!!e}>}},{<!!!>''"!>,<i<i!!!>,!,>,<!>,<!!!>!'u"!!,o!!{a!>,<!!i!!!!,"a''>}}}},{{{<!!!>a'ao!>},<!>,<!!!>e!!a!>>,{}},{<'a!>,<!!,<>,{}},{<u!!!>},<!!o!!{o!>},<{!e!!{}!>},<!>,<!>>,{}}},{{<ia<e!'>,{}},{<!>>,{}},{{<'uuu,u!>},<a!!!>!!!>'i!!!>>},<!>},<u!>,<!>},<"u{'!>,<>}},{{{{<!!!>!!!>}''!!!>!>,<>},<!}u!>},<>},<<!>,<!>!>},<!!!!!!'">},{{<u!>},<,ao!!!>,u">},<,o}ui}!!!!!!!>>}}},{{{{<!}a!!!!i"u,o!}>,{<o!!!!!>a!!{!>,<!!u,"!!!!}!!>}},{},{<!!ae!!!{}{!!!!!>},<o!>},<!!!!,!!!>!u"!!!!>}},{{{{<!>}a!>},<i'!!!>,!!!!!>!!!>,<ia!!!"o!!!>},<{}>}}}},{{{<a!aua>,{{<o<{i!>},<!>},<!a,,!!!>!!!>!>},<!"!!o!!!>!>},<}>}}},<e!>,<"!>,<a<,o''!!!>!!!>!!i,o!>},<!!{a>},{<<o{'"eo!>,<<ee,,aue>},{<>,{<!!!>},<!>,<}!!!!!>i{!!!>!>},<!>},<a>}}}},{{{{<!!}!>},<!!au!>},<>},{<ae!!{i'!!<i!>},<,<!!!>!u,!>,<>}},{<,<{!!i!>},<'>,{<}iiuo>}}},{{{<!>,<!>'}!!"!>,<"!>a!>},<u!>!,e!!"!!>},{<o"!>},<!>},<!e,>,{}}},{<!>},<!!!>>},{<ie!>,<!{!>!!!!!!!><"'!>aa"i!>'>,{{<o<!!",!!u>}}}},{{{<!!,{>},<"a}!!!>"!"!!!>o!>!,o!!!>e>},{{}}},{{{{<!>>}},{{<!>},<!>ai!!!>!eu>}}},{<!><e}oa,}o'a!!'!!!>},<!'!!<"">,{<<e!>e{,a"!!!>!!!uiu!>,<!>,<a">}},{<!!!{!!o!>!!!>},<uo!<!>},<o!>>}}}},{{{{{}}},{{{<!!!!}<!>,<u}!><>}}}},{{{}}},{}}},{{{{<"!!!!{!>},<!o!!>},{{{{<,{o!!!>!>,<{{>}},<!!ii<}!oi!>},<<"'!!<!!!>!>,<e!>,<!!a!a'>},{{{{<>},{}},<!}u{!!<a!<ei!!a}!>,<o>},{<{}>,<"{e!!!>a>},{{<!!,,io!>},<!!!e!"!a!!!>"!!!>!!,!i"!!!>>}}}}},{{{<{!>,<,a!!!>!>},<!>!!e>,{{<!!!>},<!!o'!!!>!>>}}},<"}e!!!>a!!!>!!!>!!"!i!!!>},<>},{<!!!>}i>,{<<"',!>!>,<!!!>,!!}i!!"a!!<!>,<>}},{{{<<a'!>},<a'!>'!o"o!><!!oiu!>,<!e!!!>>}},<!>,<e!>},<!u>}},{{},{<}!>,<!!!>'o"!!<e}!>,<!!!>!>,<!>},<"}>}},{{{{{{<,!>},<{"!!!!"}!!>},{<,!>},<i<!!!{{!>{!,!>},<o!>,<<'>,{<>,{}}}},{<!!!>!!!!<!''"{!>,<i"!>,<!>"!!!>,<a!>},<">},{{<'>}}},{{{<a!>},<<,!!>},{<<}!!!>!>},<i!>,<i!!!"u!!!!e,!>},<<u,>}},{{{},{}},{},{{{<!eo',!!!>'e>}},{{<"'ao<<'i{!>>}}}}},{{{{<i!">,{{<!!"!>!>},<!>},<<<aiu!!!>u<'e!e>,{}}}}},{<aai!!<}<!!o'e!>},<>}},{{<o>},{<!!!><{!>,<!!!>!!!!!>>}},{{{{},{{<uiei!o!!o!!u}<e!>>}},{{},{}}},{<!>!''!>,<>,{{}}},{<i!!!>i!"!!uua'!>e>}},{<<!!!>{!!o!>},<,!!!>,e!!,!>},<!>},<}!>},<>,<!>},<u,i{!!!>>}}}},{},{{{},{{<e!!oi!>{>},{<'o!o>}}},{{{<ia>},{<!>!!>}}}},{{<>,<a'!>},<""{!,e!!!><!!'!>!!!!a'!!!>>},{{},{{<"'!!!,>},{}}},{{{{<!>},<ii!>,<aue!>!,!!!>'a<},>,{<!>},<"oo}!!!>"a!!a}!,u>}},<ue{"a}a<}>},{<ia<!!"i!!!!!>!!!!!>},<!>},<<ue<!>i>}},<""'>}}},{{{{{{},<"ee!e!!!!!>!u!!{}e!>!',"!!!>a>},{<"!!!>},<!>!>},<>}},{{<!"}>}},{}}},{{{<'!>},<!!a!>},<'!!!!!!!!!u,!!"!>!!}'>},{<ae!!!>o!i}!!!>>}},{{<oie!!!>!>},<euuoou!>uu''>},<i}>}}},{{{{{{<!!"{>,{{{<<"!!!>!!<!!"!!i,<>},{{},{<}!>,<!!!>u!>,<,!>,<o!>,<!>,<i!>,<o!>},<!><o!!e>}}},<ao!!!!!>e!!!>e!>,<!o!>},<a!!!>>}},{{},{}}},{<u,,>,<!>,<e!'!>,<>},{<'!u"ie<>,{<uu,!!<o!!'!u{!!u!!oo!!!>}!!<"!a>}}},{{},{<e'!!!>>}},{{<}!>u!!!>,<!!u<'!!!>>}}},{{<e,!!!>!!!>a<{!!,{!>!>},<!!!!,!>},<!>!>,<o!!!>>},<aa{!!a>},{{},<'!!e!!!>!>},<!<!>,<!>},<ooi>}}}}}},{{{{<!>!!!>,<!><i"u">,{{},<{ioai!>,<!!'!'!>,<<'!>},<e}!>},<u>}},{},{{{}},{<!!!!!>!>,<"!oa!>>}}},{{{{{},{<e'!>,<!>,<!!!>!!>}}},{{{<>}},{{{<>}}}}}},{{{<!>},<!!u!>iea!>,<o!>!!o>},{{<!!!>!!!>!!"!>,<}ou!!a!!!!!>},<','!!!>},<{>},{{},{<o!>"}!>,<!>,<{i>}},{<u!>},<!e!!,!>},<o!!!>{!!!>{,!>,<i,>,<!!<u!!!>!!!!,!>,<,i">}},{<!!!>}ii!>o""!!!>>,{{{<!>},<>},{}},{<!!!>!!!>!>!>,<ae!}!>},<!!!>!,{}!!oa!!!>},<!!!>!!!>},<>}}}},{{{{<u!!>},<!!a!>},<!!{!!!>,<}!!'!>!>},<'"e">},{}},{{},{{<o,a{<e<!!!>!!!<!>},<{>}}},{<}u!>},<"<a!>,<ooa!!!!u,>,{}}},{{{<}e!>},<>},{{<'!>},<!!!>a!>'!!!!}o}!<{"!!>}},{}},{{{}},{<!!{{<i!<ou!!,!!!!!!!!>}},{{{<!>,<'eu!!!>'!i'<!!a!>,<"'!,,,!>},<!!<>},{{<"a{!>},<!!eu!!!>}!!!>},<!>"!!!>!>,<,!eo!u>},{}}},{{<!!!!!e<!!!i!>ie"!<'!!!>>,{<!!!><>,{}}},{<!!!>{!!!>oui}{,'!!!>o!!}"!>'>,{<}!!!>},<i},<ie!!!>i!!i>}},{{<<u!>,<<u!!!>,<!>,<{>,<'<<"a!>},<!!<!>,<!!!><u!!!>!>!!!>,>},{<"!{!!o!>,<<<!>e>,<!>},<e!o'!!!!{}"!!a!!!"'ie<!!!>o>}}}}},{{{{<eeea{!>ua>}},{<o!!!!!>!o!!<!>!!!>a{o}<}o!!"uiu{>}},{{<!a}e}"!!o">},{}},{{}}}}},{{{{},{{{<!!!!}'{!!!>'<>},<!!<i<,!!!!!>},<!>!>},<"""'!"!>},<!!!>}<!{>},<!>},<!>,<!!!>},<!!!>eie{!!i,!!a!>!!!>!}!!!>>},{{<e,}i!!',!"!>"i!!!>o!>},<,<!>},<>},{}}},{{<!>,<>},<<iua",<!{>}},{{<!!!>}',!>"!>},<!,!>,<!!!>!!'!!!>!o!!!>>},{{},{{{{<!!!>!o}ii!!'!!!,o!!!>u!!ee<"i,!!!>>}}},{{<!!u'<a!!'!!,i!,>},{<!!o!>!>!!!>},<<}uo!!!>'>}},{{},{{},{{{<>},<!,!>,<>}}}}},{{{<<!>},<!!!>!!!>e{>},<!>,<<!!!>oee"!!!!{<>}}}},{{<o'iu!!!>u"}}!!{>,{<u!e<e!>,<!!!>!!!>},<!!!!!!>}},{<!!<<!>!!u!!!>i"!!oua>,{{<<!!}!!}!>,<">}}},{<}ui!>,<}"!!!!!>!!!>>,<!iu!!!!!>a!>>}}},{{{{{{<}!>,<}!>!!!>'"!!!!!>a!>,<!!!>e!>,<{>,<!>,<!!!>'eo!!"!>a{u!!e{a'!e>}}},{},{{{<i!!!!u!'"!>,!!!>{,}<!>,<">},{{<'!>},<!i'!>,<!>},<ui}>},{<u,}>}}},{<>,<{}"<{!>'!u!!!!'}'<!>!i>}}},{{{<{o!u,a>}},{{}},{}},{{<!>{!!!!!>!!!!!>!>,<{{!!>,<i!,!>},<!!e<i>},{{<{ao!!!>""a'!'!>},<!>,<!!}''!!!!!>},<,o!!!>>},{{<a!!!>'<">},<'i!!!!!>u}'!>a,"!!!!!>,<!!!!!!i>}},{{<!>},<{!}>}}},{{<!!{!!a!>},<uu},!!!<>,{}},{{<{!!!>!!"!!!!e{i!>!!!!!>,<o{a>},{{},{}}}}}},{{},{{{<,e!!o}!!!!!>,<!"i!>},<o}>}}}},{{{<!>>,{<!!!!<!>},<'{a!>!uo!,!!>}},{{<'i!!!>i,>}},{{{}},{<!!!>},<!!!>!>},<,o!!i,}">}}},{{{{<{!!!!,"!!!!,ee}<,o!!!>!{!>},<,!>,<!!<{>,<!!}!>},<o!>!u}!!!>!>,<!!!>i"o!!ii!!!!aui>},<!>e<!!!>!>a!>},<!>},<!!!>!!,i>},{<!!,!>},<>}},{{<aoa>},<,!>,<!>!>,<!>},<}!>},<!!!!!!a!!i!!'{u!!!>!>,<!>,<<!!!>!>>}}}}},{{{{{{},{}},<!>{>},{<!>{<a<iu!!"{i>}},{{<'u!>,<,!u>},{<!!!>},<aa!o!>,<"!!!>!>,<!!!>!>}a}e<>,<"!!!>}"e}eo'{e,ai>},{<<'!>,<{ao!<!!!!!>!>,<,!!!'{>,{{<!>},<eio>}}}}},{{{{},<,{!!!>},<{<}">},{},{{{<o>},{<<!!}i},}!>},<,o>}},{<ie!aa}{e"!>,<''"!!}'{u<>,{}}}},{{{}},{{{<!>,<!!{!!!u!>},<!!!!<!>,<>,<!>,<!>},<i}i<e!o!{!>,<!!!>!>>}}},{<!>},<!!!>!}!>o!>,<!>},<o!!{!}!!a{!'eu}i>,<,i!!!>},<euee!!io"!a"i!!!>!>},<i>}},{{{<!<!'!>!{!!!!'<',>},{{<!!'!o>},{{<>}}}},{{<!>>,<'a!>},<"!!u<}!>},<!>!!!>>}},{{{{<!>!!!!i!!o'i!>!>!>!>,<>,{}},<!!!!!>,<!!aoe!>,<!>,<>},<"<!>},<!,<<!>},<<!>!!!>!!>}}},{{{{<i}<e!!}!oe'!!!!a>},<oui!>}>},<!>},<'!>},<!!i!!eu"!!!>',o{>},{{<!e{e}!!!>i!a<"!>u!!!!,,!!!>e"!!!>>}},{<{}!>,<!>},<i<a!!"u}!!!>!>,<!>},<!>,<'>,<!!!!!>!>>}}},{{{{<!{}!>,<!>,<>},{{{},{{}}}},{{<o}!>"!>},<!>,<,a}<!a!",ae{>,<!!!!,a,<,'uoo!u!>},<!o!!!!{!u{,""!>,<>},{<}e!!!>,<}"{}>}}},{{{},<!!ei>},{},{}},{{},{<!>},<!!!>,<uo}>}},{{{{{<!uo!>},<!!!>}eia!>,!>!!!>!!<{,!!!>!>,<!!!>!>,<>},{<!!!!!>!{<!>!!,>}},{{<u!!!,u!>},<>,{<!!!><"a<u>}},<!<!>},<!>,<'e>},{{{<!>},<}>},{{<!>},<'>},<}e>}},<i!!'{!!,e{{}!>e,u,e!!"">}},{{{{<!>{!!!!!>}!}!e!!!>!!"'{'!!!!!>>},{<!!!!!>>}},{{{}},{}}},{{{{},{<io,<<>}}},{<"<<!!ie!>,<o<!>,<>,{<"!>,<!!!>,<o!>},<{!>aai!>,<i!!!>>}},{{},{<!>},<u{e!>,<!>,<!>,<!>i!!!>!}!!!>{i!>,<!>},<ie!!!>>,{<'!!!>}!"e!!!>u!!!><!>},<!!ee>}}}},{{<>,{}},{<"a!>},<}}!>,<!!'!>},<!>>},{{<!>,<!!!>!!!!!a{!>},<!>,<!,!>},<!!!!<io!!!>'!>,<e"!!'o>}}}},{{<,'u>},{{<!!!>!}"!"!}!>u!!o>},{{},{<!,'!!!>,<!>},<e!<!>!!{!i!'}}!!'>}}},{{<}"!>,<!>{!'!>,<!>},<i!>"}!><<o>,<!>!!!{!>},<!<!>},<,a!>!>,<!!e'!!o,<">},{}}}},{{{<!'!!!>},<i>},{},{{{<o<uauiii"ea!!""u!!eu>},<"'">},{{<,e!!!>'<>}},{{<a!!!>,<'}<!!i!e!ae!!!<"!!!>u!>,<!>},<!>},<!>,<i>},{<,,o!!'!>>}}}},{{{<!!!!,"i!o!"ao}a>},<,>},{{<{!>{>},<!!!!!>"{!!!!{!!!>!>,<io>},{{<>}}},{{{{<iee}!!"!!!>!>,<aee"}>},<!!}ue!!e!>!>'e!>!!!>ue!!<!!<">},{{{}},{{}}},{{<e!u'<e!!!>e!>!!!>e!>},<!!!!!>},<!!!>'"!>,<!>,<<!!!>},<>,{{}}},<!>},<}!>!>'<!>,<!>},<!!!!!>e!!i!!!>a!>,<}'>}},{{}}},{{{},{<!!!>,<'!!,<'e!!'!!!>!!<"ei!!!>!'{<>,<o'e!!!!!!a!!ea!>!>'>},{<{!>a!>,<e!>},<e,}uioe}ao!>},<!>!!'>,{}}},{{<!!!>!!!>!!!>!!!>>},<!>,<a!!e!!!!!>,<!>!>},<!!!!a!!u!>,<"a>}}},{{{{},{<>,{}}},{{<'"}i!>,<,},<{}!>},<u"e{>,<!!!>!>},<i'}!>o{>},{<!!<!ae"e>}},{{<!>!!!>!!"<"!>u<}ua>,<'!!ou!>},<e,>}}},{{{<a{"<"!!!>},<oe!>'ie}"u>},<",ii!!!>!u{o!>},<io!>},<}!!!>!!!!!>>},{{{{<,!>,<i"<u!!!>},<oeiua}'!>},<u}>},{<>}},{{}}},{{<"a,}u!!o<!>},<!>>},{<!}{,}!!!>>}}},{{<}e<!}u!>,<e!!'ao!!<!'!>>}}},{},{{<,'eee!!!>e>},{{{<!e!>},<<ui}u!!!!!>uo}u"!!!>u>},{}}},{{<i,',!!,uua{ii!{u>},<ai!!!>!>,<!!!>!!o!!>}}}}},{{{<{}<!!<!!!>,<{"}!!!>!!!>>},{{{<e!!e!!!!i'!>},<!>!>u!!!!{ei>},{<aoao!>},<!>!>,<a!>},<!!!>}!!>}},{<!!!!!>},<!!e!>,<!aa!<!>},<!!!>!!>,{}}},{{{<'e!<i!!ao!!e,!!!e!>,<!!!{eo!>,<{>}},{{{},{}}},{{{<e!!a,!!!!'!o!>e!>'u>},<"o!'a!>,!>},<>},<a,,>}}},{{<!i!>'!iu'!!!>},<}!>},<!>},<u!!!u!!<,!>,<!!!>'a>},{{<!!,!,>}}},{{{{{{<!!!>{!>},<{}!>o,}{o">},{<'}!!o!!,!!!!!>"!>,<<!uo!ioe!!!>!>,<,!"{>,{{<!!e!!!>o,o!!>},<'o!>!>,<!!!>,<!"<e<">}}},<ao{<!!,"<!!!>u!!!!oa{a>},{<!}!!!uu>}},{<!!!>!!ii!!!>{!a!!o>}},<!>,<a"}!>},<ui"'!!{!>},<!!!!o>}},{{{<!,!>,<!>,<"!!!}!!!>,<i!!i!!<!!!>!!!"a",!!!>!>,<>}},{{{{<<u!a<e!!}>}}},{<!!!>">}},{{{<e"{!>},<!!io<!>,<!>!uoe'e!>},<!>},<!>},<'!>},<>}},{<}{ea!'"au!>},<e!>,<!!ieuu"ee!!!>>}}}},{{{{{<!"{"{!!!>i!,a>},{<<"u!!o!>},<"{!>},<!!!>{{e>,{{<!!a!>},<!>,<!>"!>,<!>},<u",,<>}}}},{<!"oi!!",!!!!!>ieii>},{{<!>},<<<}>,{}},{<,o!>!!{!>},<!o,,e"!i<"o<<e'>}}},{{{},<{!>,<i!!{!>e!>,<e!',}>}}},{{},{{<"!!e}i}e!>!>},<<!!a!'o!iui!!!!!>e>},<'!!!>!>,<}",!>,<<!e!>},<a!!!>'u<!>,<e>},{{<""!!}"!!!>{<>},<{i}!>},<a{!!ue}!>}{!!!>i!!!>o>}},{{{<u'!>,<<!!!!"!>,<"<!!!!!>},<!>>},<e!{'!>},<!!!>!>a"a!,">}}},{{{{<a!!!!!>},<{!>!>},<!a!!!>},<>},<!!!!'a<"<a!>},<!>{,<!{{!>},<!!e!>{>},{{{{<{ea!>},<u!!!>>},{<<!>{i!,'!!'!>,<}!>},<'!>},<!!,i!!a!>,<>}}},{{{<"!>},<"!>,"!!!!!<}"!!!>!{,>}},{{}}},{<,oe!!!>!!!>!>},<uei!!'>}}}}}}''')

6671